In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/app')

In [3]:
from vexa import VexaAuth, VexaAPI
from chat import UnifiedChatManager, UnifiedContextProvider
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25
from helpers import jupyter_stream_output
from uuid import UUID
from typing import Optional, List

In [4]:
auth = VexaAuth()
vexa = VexaAPI()
user_id = (await vexa.get_user_info())['id']

# Initialize search engines

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [5]:
import os
voyage_apikey = os.getenv('VOYAGE_API_KEY')

In [6]:
from chat import UnifiedContextProvider
from chat import UnifiedChatManager

In [7]:
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25


In [8]:
qdrant_engine = QdrantSearchEngine(voyage_apikey)
es_engine = ElasticsearchBM25()

Connecting to Elasticsearch at elasticsearch:9200


In [9]:
context_provider = UnifiedContextProvider(qdrant_engine=qdrant_engine,es_engine=es_engine)

In [10]:
from IPython.display import Markdown

In [12]:
display(Markdown(await context_provider.get_context(user_id=user_id,query="Dianna Lloyd")))

##  [37] - December 17, 2024 15:31

- [00:48] The chunk occurs in a conversation where Okafor Ikenna is engaging with others, expressing confusion or seeking clarification amidst a mix of languages and topics.
  > Okafor Ikenna:  Hã?

##  [15] - December 18, 2024 11:01

- [04:56] The chunk is part of a discussion between Olya Immigration Services and Olga Nemirovskaya about marketing strategies for attracting clients. They are evaluating different options for directing potential leads, specifically considering whether to lead them to Vlad's personal blog, a Telegram bot, or other platforms as part of their advertising strategy.
  > Olga Nemirovskaya:  Так, реклама.  Тут, честно, я просто не знаю, куда лучше вести, потому что у меня вариантов три.  Первый вариант – это вести на личный блог Влада, например.  например, но это Владу будут писать, опять же.  Второй вариант – это ввести на какой-нибудь Telegram-бот, где они могут какой-то тест пройти и так далее.
- [00:00] The chunk is part of a conversation between Olya Immigration Services and Olga Nemirovskaya, discussing plans for a meeting to review ideas and tasks related to their marketing strategy, specifically using Notion for organization and planning a call with Vlad later in the evening.
  > Olya Immigration Services:  Мы минут на 15, да, мысли просто обсудить, показать вот эти вот, то, что я...  Вот показать то, что я накидала в Notion, и там, наверное, если все более-менее нормально, то, наверное, с Владом лучше потом уже вечером созвониться.

##  [27] - December 03, 2024 10:23

- [00:00] The document contains a user's inquiry about the user flow and includes a test entry to assess the functionality of the system.
  > :  Делаю тестовую запись. Для того чтобы понять как он пройдет сейчас.

##  [31] - December 18, 2024 09:57

- [54:36] The chunk occurs at the end of a conversation among colleagues discussing advertising metrics and data analysis, where one participant expresses gratitude and says goodbye to another participant named Dmitry.
  > :  Да, спасибо, очень приятно было познакомиться, Дмитрий. Пока, до связи.

##  [30] - November 20, 2024 16:40

- [68:26] The chunk is part of a conversation between Dmitriy Grankin and Artem Puzik discussing the integration of metadata and context in messaging systems, specifically how to manage context in conversations and ensure accurate responses from a language model (LLM) based on the provided context.
  > Dmitriy Grankin:  В summary эти метаданные, которые много контекста содержат, информация о том, что происходило на встречах, они подаются вместе с messaging, а именно мы и это запихиваем в LLM-ку.  То есть, если контекст меняется, то мы просто динамически вот этот контекст, вот это контекстное сообщение, мы его динамически управляем а переписка сохраняется понимаешь я понял смотри просто если так  взять по-русски я говорю что кто главный спикер он горит кита там два товара  most important, но это он должен ответить на вопрос, кто основной спикер.  То есть он...  Но задай по-другому, если  ты сомневаешься. Окей.  Тогда вот здесь проблема в том, что...  задаешь контекст он может ответить на этот вопрос уже не имея контекста переписки понимаешь поэтому нужно задавать такие Вопрос, чтобы что, чтобы на вопрос, чтобы что он тебе ответил корректно.

##  [16] - November 25, 2024 13:11

- [00:34] The chunk is part of a conversation where Dmitriy Grankin is expressing confusion or curiosity about the current situation or events occurring during a discussion or meeting.
  > Dmitriy Grankin:  Что такое происходит?
- [00:39] The chunk is part of a conversation where Dmitriy Grankin is discussing the progress and status of a project or task, expressing a hopeful sentiment about collective capability and success.
  > Dmitriy Grankin:  Пускай мы все и сможем.

##  [4] - December 06, 2024 12:05

- [02:02] The chunk occurs during a discussion about the features and capabilities of a transcription and summarization tool, where Dmitriy Grankin inquires about the underlying large language model (LLM) being utilized for the service.
  > Dmitriy Grankin:  What LLM are you using?  It's charge GPT-4-0.  Yeah, something like this.
- [10:35] This chunk occurs towards the end of a conversation where Dmitriy Grankin discusses the technical aspects of a transcription model and its integration with a large language model, specifically GPT-4. He reflects on the flexibility of deploying local models and expresses encouragement for the growth of the user's project, wishing for significant user expansion and financial success.
  > Dmitriy Grankin:  And do you send all these recordings and all the data to just to...  The transcription model, which is a waste tax, is a local model on my own.  And the large language model stuff is GPT-406 for now.  But basically, you can switch with one line of code.  You can deploy.  local models, wherever you want, you can switch to them, it's not a problem anymore.  Yeah, yeah, yeah. No, really good man, congratulations. It worked. So it's, I wish you, I wish the  Thank you.  Transform those 300 euros into 300,000 and then 3 million and then 30 million, you know.

##  [2] - December 17, 2024 04:29

- [00:00] The chunk is part of an informal conversation between Marcus Torrisi and Dianna Lloyd, where they greet each other and express casual sentiments before discussing various topics related to their personal lives and work, including travel plans, renovations, and HubSpot-related projects.
  > :  Thank you.  you  you  Hello.  Hello, how are you?  Wow.
- [06:40] This chunk is part of a conversation between Dianna Lloyd and Marcus Torrisi discussing their experiences with HubSpot, including Dianna's reflections on her husband's past income, changes in farming, and updates related to their HubSpot partner business.
  > Dianna Lloyd:  I think what Andrew said, He, when his, when we first got married, he was making 40,000 a year and supporting the two times the times have changed, hasn't it?  Farms have changed it.  I saw just now, I just logged into HubSpot and I can see people and things.  Okay.  I don't know what happened because I did it last week, but I must have not saved it.  Or I tried to copy a permission set and it didn't work. But yes, I just, after looking at your email, I was like, oh, I'm pretty sure I did it.  So I've updated it now.  I won't make a comment about like, you know, that I own a partner business for HubSpot.  I own a partner business.  I'm a HubSpot partner.  I'm not a partner.  You own a partner business.  No, I would not say that.  That's what this is.  You're a HubSpot partner business, agency.  Yeah.  Or you do digital marketing that does part HubSpot.
- [03:50] This chunk is part of a conversation between Dianna Lloyd and Marcus Torrisi discussing the weather conditions in rural New South Wales and Sydney, as well as Dianna's ongoing renovation work during the hot weather.
  > Dianna Lloyd:  so it's hot but you can deal Yeah, well, I'm sitting in the air con here, so it's all right.  But yesterday and today, it's been – yesterday and Sunday was pretty hot.  How about Sydney? Is it hot in Sydney?  it's yeah and it's sticky it's like uh you have a lot of really high dew point days so excuse me i'm just gonna roll this up okay i'm having my yeah you're good Yeah, it's been a bit stinkers and I'm trying to do some in the evenings, do some renovation work in a little time.
- [02:57] This chunk is part of a conversation between Dianna Lloyd and Marcus Torrisi, where they discuss their experiences and plans while in rural New South Wales. Dianna shares her excitement about visiting local attractions, including a licorice factory, and describes the accommodations and environment of the property where she is staying.
  > Dianna Lloyd:  on slack oh it's like okay well you got the monkey cristo house in juni you've got what else have you got you've got the monte cristo historic homestead you've got the rail museum licorice and chocolate factory yeah it's so good we're going there it's so good okay so if they make a firm licorice i am like not salty but just like a firm licorice i'm down yeah i think there's nothing worse on the whole planet than salt licorice it needs to be borderline not usable stale yeah well  this place is so good so i'm sure they have firm licorice um but yeah it's uh it's going good i'm getting lots of nice food and five-star service here Here at the cottage.  And they have a little cottage in the back.  So I'm in here and then the rest of everyone's in the main house.  Okay.  Yeah, yeah, yeah.  They do have a big place.  Yeah.  Yeah.  wow it does it looks like a little um like a place to retreat if you need some time that's right so  it's like it used to be like the old brick like the old brick makers in the city they used to own this house clearly because  there's about a million bricks here but yeah it's really  they have a massive backyard as well that's cool
- [04:54] The chunk is part of a conversation between Dianna Lloyd and Marcus Torrisi, where they discuss personal renovations in Dianna's home, specifically the installation of a Japanese toilet seat, the challenges of home improvement while raising children, and the financial implications of DIY projects versus hiring professionals.
  > Dianna Lloyd:  tiny toilet just so that we can remember i told you we bought the japanese toilet seat or my parents bought it for us right because we begged on about it so much and we've all talked about it for so much um so my parents got us one and to install it we had to install electricity and oh wow yeah it's like a full-on i'm gonna have the latest thing going on right so anyway um so but is it a whole is it a whole new toilet is a whole new toilet though  or is it just like the seat just the seat yeah okay in a remote  you'll go to Japan and you'll come back and be like Diana give me the name of that  that's right it's amazing  but yeah so we're gonna we're gonna install that Christmas Eve, but I have to have the room ready for it.  So we've chased in and bricked, we've chased in a new line for the wall to put electricity in.  My husband's an electrical engineer, he can run cables and stuff yeah and so i've done the plumbing and i've been doing plastering and getting ready to paint so there you go i put in a new ceiling so it's busy but yeah we've been I've been renovating this house for 20 years and it still looks like crap.  It's a long time.  Why, why, why so long?  Because we're, like, had kids and...  Yeah.  ...money and...  Like, you know what I mean?  It's just like, you wouldn't believe how much it costs to hire somebody to do like the most basic task.  So we just go, we can do it ourselves and then it doesn't get done, right?  And indirectly, you end up spending more money on doing it yourself than you would have.  Maybe.  Maybe.  Yeah, yeah.  Well, I stayed at home raising the kids for a few years too, so that's right.  just threw things back and all of that.  Like, I mean, I had my own business, but like I wasn't making a ton of money.  Yeah, yeah.  Yeah, so it's just more of keeping my hand in a few, you know what I mean?  So, but yeah. That's a lot, yeah.
- [01:11] This chunk is a conversation between Marcus Torrisi and Dianna Lloyd, where Marcus shares his experience of being in rural New South Wales, discussing the property he is staying at, his plans for the week, and mentioning a trip to Junee, known for its licorice factory.
  > Marcus Torrisi:  what's up oh nothing much i'm in uh beautiful rural new south wales and it's stinking hot i'll send you actually i'll send you some photos oh yeah are you on a property We're on a large property, but we're not on a farm.  The property that we're on is quite big, but they don't live on a farm.  have like all the other family and stuff are on the farms let me um just the commune  yeah let me it's such a nice house that they um  This is your girlfriend's parents.  Sorry, say that again.  This is your girlfriend's parents.  Grandma and auntie live here.  So.  it's um yeah it's really a nice day like i'm working did a little bit of work yesterday morning and then working kind of today i'll probably log off probably after this call a little bit Wednesday, we're going to Junee.  I don't know if you've heard of that.  Yeah, where the licorice factory is and where a bunch of other families are.  live which is gonna be cool and then uh friday probably do a little bit of thursday and friday do a little bit of work those days and then yeah come back i just i just sent through those photos have a look pretty uh where did you send them
- [30:50] This chunk is part of a conversation between Marcus Torrisi and Dianna Lloyd discussing the complexities of data migration within HubSpot, the challenges of multi-object imports, and the importance of understanding the platform. They also touch on the scheduling of a boot camp and the sharing of documents with clients, highlighting concerns about intellectual property and the value of their work.
  > Marcus Torrisi:  good because they give you an understanding of how everything works but you know their data migration I talk about okay so you've got a contact list so you need to import it this is how you fix it and this how you import it but when you do an actual migration it is a pain in the ass so this would give you a good idea but then yes you'll eventually have a situation where you do need it import like multi-object like the multi-object ones are really hard not hard but like that complex, you've got to get them right, like the first time.  Otherwise, yeah, we've, I've had, I've had things where, uh, they weren't sinking.  And so they were getting skipped and there were errors and you have like limits.  they have to throttle it yeah yeah so I'm gonna watch that but you know that the things that I've done in the past were were multi-object but I was also with the data team and we had a custom API key and like you know they were handling sort of the push yes and I was handling the mapping and the projecting of you know so awesome So anyway, it'll be, I think it'll be a good one and it'll help me, you know, I think it's also one that's like, if you can understand that, then you can pretty much understand the platform.  Right.  So.  Agreed.  Yeah.  yeah there's a few more i thought that were good but like they'll take the time uh and i also signed up for the boot camp boot camp are you gonna do what you think well i do  I looked at your link that you sent, but they're at 2 and 3 a.m., and then it didn't say anything about recordings.  No, you got to go.  Once a week for six weeks at 3 a.m.  I am going to bitch the entire time that they need a Southern Hemisphere version, or they need like two time zone versions, you know, that I can maybe choose a late night version versus a 3 a.m. version.  And they shouldn't all be based out of U.S. time.  No, no.  Hello, kitty.  My cat's looking very hot.  So did you hear from Yorder?  I heard from Yorder.  Their book, they literally just emailed about 10 minutes ago um we're gonna do 10 a.m on thursday okay cool so you'll message me okay yep I'll create the event for that.  But yes, I think good, um, good idea there, like to, you know, to walk through, we're going to walk through that on Thursday, obviously, but I think, yeah, one of the key things is obviously it's a simple.  buy it as much as possible oh you've already done that awesome yeah sweet i've created that  i probably simplified even more but like i feel like these are a lot smaller um i'll fix that one that was the one i needed to go back for this one here is what I wanted to finish up but for the rest of them they're a lot shorter now yeah that's awesome yeah  cool  I mean they're  probably still  long in your opinion but it's not  Like it's not about, um, if you can, if you can talk about them in a more succinct way, because what we obviously do is we obviously share this with the client, so like if they want to go back and, and.  So you do share this document?  With clients that I think are not going to steal it.  Uh-huh.  Which, so, like, I don't, well, yes.  I don't like to share it because like I said, and like I say, you might have heard me on the call with Yabba. I was like, I'll share this with you, but just keeping in mind that this takes a lot of work and it's not really it's not really expected to be shared with anyone else.  And that's what I try to include on the email as well.  But there was this one client that basically said to me after we did an audit, probably about six months ago, but he was like, oh, look, you know, you, you guys are quite expensive and I can just get an offshore team to do all these things for me.  And that's what I feel like this is like a technical SEO audit.  If somebody gave this to me for free, I could just have my team run it.  Yeah, that's fine.
- [25:07] This chunk is part of a conversation between Dianna Lloyd and Marcus Torrisi discussing the use of documentation and communication tools for managing client interactions and project workflows. They explore the effectiveness of Google Docs versus Slack for tracking questions and answers, the importance of understanding HubSpot's lead object, and the need for better organization in their work processes.
  > Dianna Lloyd:  Um, let me think about that. I think you could set up like a notification or something like that. But, but I just the idea, it doesn't have to necessarily be a Google Doc. But I mean, as soon as you tag somebody off.  Obviously, that works.  Yeah, yeah, that's true.  But let me just have a look and see if I can find it.  We can talk through it and see if there's a solution that works for us, because I was just thinking, you know, this is going to get bigger than just you managing, right?  And so you're going to need to see what I do and what, and I think just looking at my tasks is not always going to show you what the questions have been and what has worked on yeah you know and i and i think i think it's probably a good idea i've never had to work with a document like that other than with the hubspot csm yeah but it was good it was really good i found it good i found it helpful i was always using it we were back and forth in the con you know and she would go out and get answers and come back to it when she wanted you know what i mean but it was like this little bulletin board for the two of us like a slack channel i guess in a way but slack doesn't hold on to data long and that's the problem with slack is your your account i think clears after 90 days you lose you lose information nation no not not not ours for me it said it it only holds 90 days of history  does it still say that now because it Shouldn't, because I'm paying for you as a user.  It said that to me before.  I'll check.  So are you sure you're...  Yeah, I've got all...  All time?  Yeah.  I've got all my yeah all time. Yep. Okay, I got a message that mine was 90 days  So I'll just double check that that was a couple of days ago. I thought that was last week. I Was looking for something and it said 90 days maybe it's a setting you've got for me no it's that i need to upgrade  no you're a regular member so you're you're a paying member That shouldn't be the case, but...  Oh, look, okay.  Yeah.  But it's probably not the best way of keeping your history of everything anyway.  Yeah.  No, yeah, because everything is – yeah, I'm paying for it, so it should be there.  Oh, one – one – I mean, we could –  That one is something like this in Slack, but they have to be on Slack.  And I think Google's might be better because yeah, there's like channel and all of that and Slack, you're going to get all over the place.  And it's nice to have like just like I've got one client you know what i mean yes yeah so um okay and then i've got a 430 and then um We can talk about.  I've gone through it, but like at some point I'd like to love to have a coffee and just sit down and like, or a beer or something like that and have a chat about some thoughts I did.  Like I said, last week.  spent a lot of time learning about the the lead object um and how it might be used and i think i have a much better understanding of using it moving forward i think it's not I think it's something that people are kind of ignoring.  But I think HubSpot are going to be pushing that it gets used.  Yeah.  Well, it's relatively new information.  Yeah, it's new. It's been the last six months, I think, or a year.  It's only, like, it came out, I think, like, April or May this year, so...  Anyway, I went through a couple of videos and watched and I got like, you know that one of them was really talking, I grabbed this from one of the videos and I was like, Oh, this is really cool.  And it was, you know, talking about the different life cycle stages and how they saw it mapping and how they work together.  Yeah.  And so I thought that was a useful...  way of to help somebody understand how these different things work together because i'm sure that clients probably really don't understand that no they don't  I find it hard.  I actually think that they, and then you've got like this life cycle stages, and then you've got contact life cycle stage, and then company life cycle stage, which I'm assuming they'd have to be absolutely  completely married yeah they're synced yeah so they're synced on the back end so  okay anyway that's in there so whenever you want to go look at it um So, yeah, so still doing the HubSpot Academy stuff.  If you have anything that you would like me to do, I've started with this one.  Yeah, awesome.  Love it.  It's a big, chunky one.  Yeah.  That's a good one, yeah.  Have you done it?  I think I've done it at a previous – when I was working for a first job.  Pretty sure I did the certification, yeah.  Okay.  done it in real life yeah same yeah and this is the thing it's like the certifications are really
- [22:08] This chunk is part of a conversation between Marcus Torrisi and Dianna Lloyd discussing their communication methods with clients, specifically focusing on the use of email versus collaborative tools like Slack and ClickUp. They explore the idea of using a shared spreadsheet to manage client interactions and track questions and answers, highlighting the importance of maintaining a living document for effective client relationship management.
  > Marcus Torrisi:  little page but we don't need to anyway okay um i just saw your question and pending item around samantha can i added to the slack channel to connect with her yeah um we we don't have a slack channel with taurus oh okay  over email okay so it's just all over email and stuff why don't we have one they don't use it  they i am unaware if they use it i've only got two two connections like members And then I've got one of my accounts, Gather, we've got them and that's it.  Okay.  the cleaning that's the only they're the only ones i saw that okay okay yeah and then there's like binnacle training or binnacle but we don't really use it i just email and call the guy that we work with.  Okay.  Do you have a document then that you work with out of with a client that is always like a spreadsheet with like lots of tabs and it might have a place where they can put questions or are you just operating out of emails?  You're usually just operating out of emails And then based, and then based on the task, I would just create the task.  In ClickUp.  In ClickUp, like ClickUp's my source of truth for things that we're doing.  Okay.  So one of the things that was a.  really good tool that and i'll try to find i did save a copy of dives version of it but die walsh uh and i think this is something that hubspot does she has a spreadsheet right and i don't know if you've ever worked with the CSM, but what she does to manage that client relationship, which I thought was a really good tool, she has this spreadsheet that she shared There's between the two of us, right, a Google Sheet.  And it has one of the tabs is basically question, answer, comment kind of thing, right?  So, and it keeps like this living history between everybody can go on there and add things.  And then she has like, we had our weekly cadence or our monthly cadence or whatever it was.  her to have time to go and find the answer.  And it was like, is it urgent?  Is it, you know, it was a really nice tool that you could have with clients that I think gives them a place that they're thinking of something they can  pop it but they don't have to necessarily bug you okay so then how i i i've i've done not done something but i've been involved in something similar like that before i'm pretty sure it was with HubSpot how do how does the CSN know or the person receiving the question know that there's a new you can comment well you'll get a new task
- [14:10] This chunk is part of a conversation between Dianna Lloyd and Marcus Torrisi discussing the use of a free CRM for a startup's website, the tracking of emails and meetings, and the preparation of workflows and documents for client interactions. It highlights their collaborative approach to managing client projects and ensuring effective communication and documentation.
  > Dianna Lloyd:  Yeah, I'm using one for network code. It's a free one. And it's really limited. It's very limited. And I'm, I'm shopping right now for another customer of mine. And I've told her build her her website on the c3 crm because she has she's a startup and i just think it's the best way of starting as a free like even if she doesn't use hubspot i still think it's worthwhile. If you just need to have some landing pages and you need to have a presence in some forms, it's free. Like I don't see the drawback for it.  Yeah. So that's what I'm doing. Yeah. So that's what, that's what we're in the  it of it now but in regards to like logging emails looking at meetings that are against contact records or deals or companies that should all be there with your level of access i'll have a look  can make sure it's all working right so that you know because i have like meetings set up and emails back and forth i'll just double check now that i can see i'll just double check that everything's mapping correctly and all of that um  So I can see, like, everything with, like, Samantha this afternoon is tracking in there.  Okay.  Yeah, which is good.  Okay.  Cool.  Alright, so that's your plan for me. That was my question. What's the plan for me?  So I'm meeting with I think I'm not meeting with Samantha today.  I think the only one that's interested is Shannon.  So it must be Shannon.  Samantha doesn't give a poop.  Yeah, sure.  So yeah, fine, whatever.  I'll talk to her and see what She wants, and she can be the middleman for passing on the ideas.  I can really pass on the ideas, right?  Like, no, no, that's right.  Exactly.  Right.  And that's all you need to do.  Yeah.  Yeah.  And just say like, you know, based on my experience this is what i thought would work yeah yeah exactly and if they don't if they don't take it on will you you go well here's your here's your workflow is your workflow here's  your email is it ready to go you want me to turn it on all right okay  yeah so don't yeah don't don't stress about it too much if they don't take i'm sure she'll take on the feedback because you've got good ideas but like if she goes nah we want to do this you just have to go yeah all right we'll just do what you want  You'll see in the document or the e-mail I shared with you, probably had a link to the Taurus sales contact form journey.  Yeah.  You may or may not be interested in using this is sort of sort of document or if you have something that's different that you want to use with clients to sort of walk them through a workflow and this is actually two workflows in this document because it helps better explain.  This is what I'm gonna walk her through when we go through it.  Like I said, you come in by this, then there's the next one.  And this is the experience, right?  So, and then we can, just walk through it quite easy and then she can you know i've got the original brief in there um  and then was this the one was this the one that it was like left to right like showing the flow of the of the workflow, is that the one that you want me to talk about?  No, I'll share my screen.  It's just a spreadsheet.  It's just a very basic document is to like kind of track what has been done and then just to walk them through it.  So like once we start building something, they can see and you can walk them through it.  This is what the customer experience is, right?  So especially when it comes to a market.  getting and you know maybe even sequences we could add in there you know and just say this is their experience to the whole thing because I think that's what happens with these people is a lot of times is they just don't see the end entire picture of the customer journey in that one thing.  And this is the version so that we could always use this document going back between the two groups.  Just saying, this is what we did.  I have to make a version two, you know, so I've got next step suggestions and things like that.  They can use this to talk to their team about what was done.  Yeah.
- [18:17] This chunk is part of a conversation between Dianna Lloyd and Marcus Torrisi discussing project management and collaboration tools, specifically focusing on organizing documents, creating templates, and managing workspace access in ClickUp and Google Drive for their ongoing marketing projects.
  > Dianna Lloyd:  I'm just trying to find a very basic, very few words kind of, way with all the links in it, the messaging's in there, the CTA, the value add, it's almost like a content calendar in a really weird form. So this is similar to something I've done in the passed just to make sure that it was very easy to see all workflows and how they tied together um yeah you might have seen sequences can be added to campaigns now which which is pretty That is cool, yeah, awesome.  What would be good is that, it would be good to link this document in side of the task um in click up yeah just so that like when we go because like you know like both of us will be in the next couple months or coming to next year There's going to be things flying everywhere.  What I usually do most of the time, if I'm sending a document to the client, I will have or pop it in the comment side on the right-hand side.  That way it's all there and all in one spot.  I think I, is that me?  Is that it?  Yeah, I did already.  Oh, okay, sweet.  Oh, there you go.  Awesome.  One step ahead.  One step ahead.  Okay.  Sweet.  Yeah, I like it.  If you want, I can build out a template if you want to use it and edit it or whatever.  Yeah, that would be good. Yeah.  Or you can make the template.  I don't have admin rights.  our workspace so you'll have to do that another thing  after  give me  admin rights  to the  workspace  can you  tell can  you tell I've ran this for a year and a half my on my own this is why you need to let go of some of the things if you want  give me admin rights oh yeah no no i know no i know i'm more than happy to do that but i'm like  i've never had to do it so i'm like i know it's okay it's okay  Still don't see the drive anyway.  So I know that you're working on that and Christmas, but, um, cause it's not going to be good long-term for me to be operating in my own personal drive with folders for clients.  Yes.  I agree.  So but that's where I'm at right now because I don't see it when I go to drive.  I'll just share with this with you.  This is what I see.  So if I see.  workspaces i don't have any workspaces i just have my drive which i've started creating folders um which is not ideal obviously and then um I don't have any folders shared with me except for one.  Okay.  I've just created a workspace.  Okay, cool.  Yeah.  If you refresh that, is that...  Show for you  your workspace  That's a add  i don't know we can deal with this another day like it's not a it's not a today thing but just letting you know i'll keep it in my my update please Yes, please. Sorry.  I'll add, how about this?  Yeah.  Create, I'm on this other laptop.  shared workspace to make templates and even maybe even like a Not a SharePoint site, but you know that they've got the the sheet the thing called Like if we wanted to create like a
- [07:48] This chunk is part of a conversation between Marcus Torrisi and Dianna Lloyd discussing their use of HubSpot, specifically focusing on the challenges and decisions related to subscription plans, access levels, and the need for a discovery session to better understand their client management processes and tools.
  > Marcus Torrisi:  oh i don't know what i am i don't know what i am we need to have a discovery session we do and next year we do a lot of things on my list of to-do's next year okay so that's that's Yeah, I could tell that your brain's already turning off, so.  Yeah, no, no, it's still on, but it's not on that stuff.  That's the thing.  It's more about, more focused on the client side rather than that branding stuff, yeah.  It's like, it's on next year.  So I can see that I have, what kind of access do I have?  You've just got a core seat.  Okay.  So you don't like you have.  But HubSpot's really given me the shits because we had to actually buy HubSpot sort of now released a new update for partners.  Partners have to spend $600 US a month on their subscription.  So why did you get content?  and not another sales seat and something else and more contacts or something okay so contacts i  okay so the reason why i didn't get more sales seat you have to have a minimum of four sales seats okay so i was like well i'm really we're going to be using like one and a half of those like i mean if you had a sales seat we can get you a sales seat when it comes time like if you're going to be doing more sales but i was like i'm not going to waste money on potentially three seats that we won't be using no i would probably be using it more like  not that I'm going through like a sales pipeline process right but more of maybe is it a service level or something like that like I want to be able to record what I'm doing and then like have meetings and email people and follow up with them and go through like you know go use the if we're going to use the tools like use the tools like the the playlists or not playlists but the uh Playbooks and stuff like that, you know for like the Sessions where I'm like, okay, let's walk through this and like make sure I ask the right questions You know, I mean blah blah blah and we capture the right details every single time times so that when we go back, if we have to hand over to each other, we've got all that information there, right?  Yes.  Yes.  So like playbooks, in that case, would be the only thing that you would need access to.
- [49:40] This chunk is part of a conversation between Dianna Lloyd and Marcus Torrisi discussing the management and cleanup of client properties within HubSpot, specifically focusing on tracking hours for client projects, preparing for a discovery call with a new prospect, and outlining the objectives and requirements for integrating HubSpot with Dynamics for marketing purposes.
  > Dianna Lloyd:  What do you think about the new thing with being able to archive properties even if they're in use of something so like no meaning like what do you think about cleaning up people's portals and like just cleaning out a whole bunch of properties that they don't use and just oh yeah 100 is that what you do for for clients do you go in and you just go you're not using Like, 60% of your contact properties.  It wouldn't be something that we would do initially unless that was what we were tasked to do.  That would be like we finished...  Like, say we had an existing account.  Let's say, like, Next Mile.  Like, if Next Mile were, like, kind of draining the amount of work that we needed to do, it would be, like, we kind of try and do, like, sort of a quarterly like review and like what kind of workshop with our clients to go okay these are the things that you want to do in the next three months if that conversation is kind of like dead That's when we would go, okay, properties, let's clean them up and let's consolidate.  Let's fix your views up.  Let's do all these things.  It's like one of my clients, Hunter Gather, who are the buyers agents and the property  guys they have like a bunch of workflows like every account that are kind of like stagnant and maybe be deleted so that's what we're doing over the christmas period to make sure we use the hours is doing that  validation piece okay and when you're tracking hours for these guys let's say you have um taurus and you've got i don't know how many hours you have with taurus  yeah it should it should say on the top so if you go in if you're going to click up you're going to share your screen it might be yeah yep  of 21 hours and 20 uh what else is four hours i can't oh my god if you go to  Now, if you go to the list on the left-hand side, so yeah, but yeah, that one.  So if you click on that, Oh, my God.  What is with this?  Do you have...  You have, well, ClickUp's integrated.  See, I have that as well, but I've got the timer.  So if I share my screen, this is so frustrating.  So see this.  I can see here.  I don't see all that.  why like this i don't know why but it's integrated with clicker it's all there it's active maybe going to configure maybe it's everybody Now the config is like the only  Members  Oh, wow. How'd you do that?  I don't know. What? What today?  If you do 20, if you just do 20, it logs in as hours.  I did, I did 20 minutes.  Yeah.  So if you have that, I've been called out so many times, but if you do that, yeah, it, it defaults to.  to hours no i'm done for the month yeah thanks for your help we'll chat next year  um yeah let me have a um just get back  me on that you don't have to it's not it's not something i have to have today i'll put it in my little list of questions of like please how do i track but you should be able to see that i don't  know what the setting is  How do I keep abreast of the client hours?  Like, for instance, there will be...  Taurus would be a very good example of I'm doing some things for Taurus and you're doing some things for Taurus and we don't over promise on either end that we'll eat into the hours total for Taurus.  exactly yeah so that's kind of what I'm thinking or it could just be in the meantime we'll just message I'll just message and just say you'll tell me you can't spend more than two hours on them if it's going to be longer than we need to talk to them and say look it'll have to be next month or whatever yeah yeah i don't like  yeah that's fine that's fine with me it's a good i mean if it comes to  Is that right?  So I'll just keep that communication open and I'll just say, look, how many hours have we got left with Taurus?  Can I do this blah, blah, blah for them?  Yeah.  But even like I say, Diana, I'm not a micromanager.  or I'm not, I don't try to be a micromanager.  So if you like to me, like, I really want to do this and it's going to potentially go me.  We're going to go over the hours by one or two.  are really big on value-based pricing so just just because we go over a couple hours it doesn't mean that we don't do it yeah no that's that's fine it was just more of like i don't want  blow out your hours on something when you didn't want to or you've already blown it out somewhere and you're like no no no they've gotten way too much this month and that and  that's where that tracker is going to come in handy so yeah exactly but in the meantime  Until we get that, I'll just make sure I check in with you.  It's not about micromanaging.  It's just making sure that at least you have some estimate of where the hours are being used for different clients, right?  And then when I get my own clients going, I should be able to see that and manage that one myself, right?  Exactly right.  Yep.  Love it.  Because I've got something in about five minutes.  Yes.  okay was there anything quickly that we need to run through  on your list yeah the only other thing  was um did you want to give me some homework are you happy for me to just continue on uh  doing the gap fills no so i'm going to give you a task to do which we've got a and i'll try and say it in 30 seconds or less one of our clients we have a new prospect richard j they're called richard j they do like industrial washing machines industrial like dry cleaning equipment for hospitals and cool businesses like that they're looking Looking to get marketing hub, not sure if there is professional enterprise and they want to integrate that with dynamics.  Okay.  We have a discovery call with them on the 7th of January.  before I'm flying overseas and having some nice sashimi and What else I saw ideas and things like that, right? So What I want you to do it all and I'll share with you some resources sources, we need to do a discovery with them, like a marketing hub discovery to understand their key objectives.  Exactly what they want to do, exactly what they need to see within HubSpot for their side and we need to get a good idea of what questions we need to ask based on um and i'll share with you the initial conversation that we had with them on firefly on fireflies yeah so fireflies we recorded the conversation google fireflies  no no just fireflies  the AI no-taker  oh oh sorry I was thinking about a totally different thing and I was like you're using that that's like getting destroyed or something it was like totally out it's old so we  We obviously need to prepare an agenda for that discovery call, okay?  In which they, yeah, you'll have a listen, but I'll provide like a better brief I can provide you with some questions and everything like that, that we may want to use, but I want to, from your marketing side, I want you to sort of put together a bit of an agenda for that call so that we get the...  objectives and we get the scope of work that we need so that we can put together a quote for them.  Because that could be a really cool project to work on.  So did you say they wanted it to integrate with something?  Okay.  So HubSpot would just be their marketing tool.  They won't use sales or any service or anything like that.  They're just going to use it for their marketing  And then once they have leads and things like that,  push them into Dynamics.  Okay.  So what questions to ask them?  Yeah.  What are their needs?  What are their requirements?  requirements what are their objectives as a business yeah so that we can put together a  scope for them and i'll and i'll send you some a bunch of questions that i got from my account  manager and stuff but it's going to be a pure marketing project so i know you've got
- [35:11] This chunk is part of a conversation between Marcus Torrisi and Dianna Lloyd discussing the implementation of a lead management process for a client, specifically focusing on the use of meeting links and workflows in HubSpot to facilitate lead qualification and follow-up by OLM (Online Lead Management). The dialogue explores the mechanics of how leads are handled, the integration of meeting links, and the communication flow between OLM and the treatment coordinators.
  > Marcus Torrisi:  But, but like that's, and that's why, and that's why I didn't share that with that person and at the end of the day we aren't going to we aren't going to be able to tell but at least we've got access to their portal so we got the managed points for that to account because we're the only partner in there so like yes it's a win it's a loss from our win but you know at least we are we're providing a good service yeah there's a couple i missed i can  Yeah, I'll go back.  I was jumping around.  So yeah, I'll finish that up.  Yeah.  That's no big deal.  And then...  Yep, talking to them today.  Yep.  Samantha, we've already talked about that.  OLM project, what is OLM?  LLM stand for? Oh, LLM is that other company? Sorry.  Online.  Yeah. Lead management. Yeah.  Yeah.  You included me in the meeting.  and i spat out a few ideas back and forth with you in this thing yes but i didn't know what happened after that like it sounded like they adopted the idea but i wasn't even sure if the idea would work Yeah, well, we went down a different route.  So instead of using the landing page form, what we've used is the meeting link for that particular clinic.  as like the so when someone because what what olm is doing they're just qualifying the lead and then they're booking in for a callback i guess the more like a more qualified call back from the treatment coordinator so so so what what they're what olm is doing is basically calling up the the lead and saying look you know we're from next mile we're blah, blah, blah, you know, I'm going to ask you a few questions.  And then once they, I guess, qualify, they'll say, oh, look, well, what we'll do is we'll book you in for a callback with one of our treatment coordinators.  Just a warm handover.  Say again?  Just a warm handover.  A warm handover.  That's exactly right.  Yeah, okay.  And what we do is we're going to use those meeting links as the trigger for them to be, I guess, removed from OLM having to deal with them.  in all aspects. Okay, so if the meeting link  is known or something like that? Well, it's like  because the contact will be submitting the meeting link, but it's actually the OLM salesperson.  So we on their behalf.  Yeah.  So we it's like, yeah.  So we basically say in the workflow is like for any form submission is the meeting link, because HubSpot understands, okay, the contacts submitted it, but they haven't submitted it, but they have, but they haven't, so then that's we use as the trigger and the work.  It's a HubSpot form, right?  Well, yeah, it's like a, it's like a meeting link form.  Yeah.  Okay.  Sorry.  I'm just, I know that you don't owe me this explanation, but I was trying to understand, so the meeting link.  itself yeah is how is how are they where's the meeting link sit on hubspot no no sorry the customer has called Contacted Smiles.  Yep.  OLM gets the form information somehow to recontact this customer.  So they get sent an email or something?  Correct.  Yes.  Yep.  They get sent an email that says.  Hey, such and such details, because you've personalized the email that goes out and it just sends it to OLM.  OLM says OK, so it goes in their queue.  They get in touch with the person.  The person says, yeah, I'd like somebody to call me back.  What does OLM actually fill out?  The meeting link form.  Where's that sit?  They just get.  Oh, it's just a URL that OLM has, and then they just fill in the details.  Exactly right.  Oh, okay. So a form is created And you can even put in, like, I guess, a hidden field saying created by OLM or something like that.  Well, yeah, that's right.  Yeah.  Okay.  Okay.  And then...  Off the back of that form submission on the meeting link.  That's what we do with our triggers.  Okay.  So you just created a form in HubSpot and given OLM the URL.

##  [18] - November 25, 2024 15:20

- [05:48] The chunk is part of a conversation where Dmitriy Grankin expresses frustration and confusion about a technical issue related to registration, criticizing an individual named Андрей for causing problems.
  > Dmitriy Grankin:  Очень умный, очень умный долбоёб.
- [12:20] The chunk is part of a conversation by Dmitriy Grankin expressing frustration and contemplation about a project, specifically discussing the need for refactoring and the challenges faced, amidst a backdrop of technical issues and dissatisfaction with the registration process.
  > Dmitriy Grankin:  сбираться, оно мне нравится, очень сильно мне нравится.  Сюжет, думаю, идти мне рефакторить или не идти рефакторить.  пойти за рефакторе потому что это просто какое-то просто издевательство сама  самобичевание вопросам хотя работала конечно все вроде  В любом случае, если ты чувствуешь...  Это дело, наверное, много причин и...
- [03:51] The chunk occurs during a frustrated monologue by Dmitriy Grankin, where he expresses his discontent with a problematic registration process affecting users, highlighting the challenges faced in managing the situation.
  > Dmitriy Grankin:  Ужасная сила.  je vais passer au processus des speakers Tak, zaspikiwaj mnie.  Бедные, бедные мои пользователи.
- [04:42] The chunk reflects Dmitriy Grankin's frustration regarding issues related to user registration, specifically blaming an individual named Андрей for the problems, amidst a broader discussion about technical difficulties and user experience challenges.
  > Dmitriy Grankin:  А всё из-за регистрации. Всё из-за чёртового регистрации.  Из-за долбоёба Андрея, который И вот такое вот устроено лоховство.

##  [23] - November 25, 2024 13:44

- [00:06] The document appears to involve a conversation or discussion about difficulties in finding information or understanding something, with the chunk expressing frustration about the lack of convenience and clarity in the search process.
  > :  ...какого-то удобного, но ничего найти нельзя, вообще ничего не понять невозможно.

##  [21] - December 03, 2024 18:40

- [01:04] The chunk is part of a conversation between Sergio Goriachev and Dmitriy Grankin discussing the challenges and mechanisms of liquidity provision in international trade using DeFi tools. They explore the complexities of transactions between suppliers and buyers in different countries, emphasizing the need for expertise and liquidity in logistics and financing.
  > Sergio Goriachev:  Короче, не совсем даже токенизация самих товаров, это когда-нибудь потом, но меня интересует налив ликвидности через DeFi-инструменты, то есть, допустим, вот...  Как это работает? У тебя есть поставщик с товаром в одной стране, у тебя есть покупатель с деньгами в другой стране.  И у них вечная проблема, что поставщик...  Отставщик хочет отдать товар вот здесь, в своем порту, а покупатель хочет заплатить за него вот здесь, в своем порту.  И обычно нужно очень много ликвидности на логистику, страховки, на веса.  товар, и здесь получается такое плечо инвесторское.  То есть, допустим, 5 миллионов долларов загружают, чтобы чуваку здесь заплатить, а товар здесь отдать и заработать на этом процент трейдерская операция наш клиент как раз и эту ликвидность не всегда очень просто найти То есть это должен быть человек с экспертизой в домене, потому что никто просто так не даст деньги на руду марганцев.  из Бразилии в Россию, ну это crazy, нужно быть.  Вот, а...  Да, а...  Самое удивительное.  Да, а через

##  [11] - November 25, 2024 10:13

- [00:00] The document is a test recording aimed at evaluating the recognition and recording of speakers, with the author monitoring the process for effectiveness.
  > :  Делаю тестовую запись для того, чтобы посмотреть, как будут спикеры распознаваться, записываться.

##  [35] - December 04, 2024 15:32

- [00:00] This chunk is part of a conversation between Jasper Vanuytrecht and Dmitry Grankin, where they discuss their current locations and the weather while Jasper is on vacation in Catalonia and Dmitry is in Cascais, Portugal.
  > Jasper Vanuytrecht:  how is it going good I'm spending some days at the coast here in Catalonia  what's the weather like it's a little bit cold uh like well  cold you know 10 15 degrees oh yeah but it's funny and you're in cascades uh yeah not far  It's about 20, I think now, this morning it was 16.  Okay, and it's Cascais, right?  It's 20 kilometers from Kashkash.  Okay.  If you're familiar with the area, I can be more specific.  Okay.  I've just been to Lisbon yeah it's it's about the same distance from Lisbon to like 20 kilometers

##  [7] - December 10, 2024 11:01

- [08:30] This chunk occurs during a conversation between Waqas Arshad and Dmitry Grankin, where Waqas discusses the importance of SEO and content marketing for Dmitry's product, which is currently in beta testing. He compares Dmitry's product to competitors like Otter and Firefly, emphasizing the need for a blog section that addresses user queries to enhance conversion rates.
  > Waqas Arshad:  So as you know, you are actually launching your product, you are beta testing it right now.  People are getting traction and obviously from product and we can see that people are actually giving good reviews about the product.  So that actually shows that your market fit right now.  So once you go towards the launch, you should actually start working on your SEO and SEO should be focused towards your content marketing.  your blog section.  So what I have to say about your SEO, I mean, the main reason I reached out to you as well was, if you have seen your main competitors like Otter or Five Flies, Just I would like to know, do you consider Otter or Firefly as your competitor, or do you have some other competitors that you think are more relevant, if you can name them to me?  Yeah, we've got Otter, Firefly, We've got Fathom, we've got ETLGB, and these are the main competitors.  Okay, so while doing my head  I went on to Fireflies and Autumn.  So if you actually go onto the website and you did a little exploration on the website, you will see that both of them have a specific blog section on your website. And if you actually go a little bit deeper and see what type of content they're actually writing on the website. So that's not just an informational content that they write, their main goal and their main purpose.  was for having a block section is to address the queries and address the problem their users are having so that they can actually convert and start using the product, right?
- [30:22] This chunk occurs during a conversation between Waqas Arshad and Dmitry Grankin, where Waqas discusses the importance of targeting interested users and converting them through high-quality content. He proposes a trial for content creation to demonstrate the value of their services compared to cheaper alternatives, emphasizing the long-term benefits of quality over quantity in SEO and content marketing strategies.
  > Waqas Arshad:  Their major focus is to only target the users which are interested and then convert convert them. So based upon the services and based upon the value proposition that we have, I believe our pricing is very much cost effective. And as for the main leaders in the market,  yeah okay i have a proposition for you by the way  you can try out i mean the people who are cheap in the market in the market and just try one article with me you can do a trial with me we can do paid trial with one article alone where we do a topic selection for your brand and write the content and then you can publish it and you can try provide some other chief agency as well and you'll have a benchmark of the quality that they are providing and what we are providing and then only then you will be able to actually feel the difference in what we have to offer and what other companies who are charging less are happening to you. Otherwise, it's just discussion that we're having. Maybe you can simply say he's just making blame excuses. He's just making claims and they're not doing something different. So only you can only figure out once you out once you try and benchmark it with some other service that are cheap i would say do a trial with me and do a trial with another company and then you can figure it out yeah okay  Okay. Okay. We'll get back to it. By the way, do you use Google Meet as your primary meeting app?  Yeah, we use Google Meet a lot, but we're not only using that.  We use Zoom as well.  We are using Microsoft Teams as well.  So, it's totally based upon what is the preference from the client.  And do you say any of the note takers available in the market?  As of yet, I'm not using any of them, but I do plan to use going forward.  Okay.  Um, so the use case I've been, uh, you have a non.  this specific call with you now is that I've been I've been checking the numbers Four pieces of content, 8, 6, 22, and the price is something I've missed by ear.  and that's something that I have and now I can do something like summarize the proposition So the proposition is content creation focus on high quality competitor analysis is your strategy.  the proposition the pricing I am so so that is uh something that can be done real time during the call and there's much much more after the call that's come down so So I encourage you to try it out, too.  It's free.  Yeah, definitely.  I can give it a try and see what we can do.  uh uh but what are your thoughts i believe you found the pricing to be very high as for your  budget right yes i i was feeling like uh is that like 150 bucks per article is something considered and considered pretty high  there was a time when i used to feel the same but once i get into this and the quality if you focus on quality There are companies which charge $1,200 for just one article.  There are companies in the market.  The main thing is what actually you are targeting at.  You're having a SaaS product.  and you want to make sure that your SaaS is recognized as an authority in your niche.  In order to do that, you need to actually focus on quality.  If you actually compromise on the quality and go towards quantity and go towards cheap SEO and cheap agencies, it will definitely, uh, fired back down the road because for one instance, you try a cheap SEO and you focus on quality quantity and you publish like 50, 80 or a hundred content.  10 pieces in a year or in like six months.  And then you actually realize, okay, it's not working out.  I'm actually getting some traffic, but it's not giving me any benefit.  We were at the end of the day, a million traffic on a website.  Like Alexa.ai is not going to benefit if it's not converting, right?  1000 traffic, which converted more value.  If you want to make it right at that point of Time it's going to cost you double because 100 content pieces that are already published should be revised again. A new SEO company will come and will do a complete audit on your content, will figure out what topics to retain, what topics to remove, It will take time.  Then they'll have to update the content and it's gonna cost you twice the money that you're gonna do it if you focus just on quality.
- [25:26] This chunk occurs during a discussion between Waqas Arshad and Dmitry Grankin about content creation strategies for Dmitry's AI tool. Waqas emphasizes the importance of quality content and the limitations of relying solely on AI for content generation, while Dmitry expresses his interest in potentially outsourcing content creation. The conversation highlights the need for a balanced approach that combines manual processes with AI assistance to ensure effective content marketing.
  > Dmitry Grankin:  need to dedicate some specific time for doing that the other idea is that you are suggesting is that just fully just fully fully outsource this thing or if that's a full outsourcing okay just uh give me an idea what what's going to look like okay because otherwise i have a clear idea how to uh how to proceed doing this myself in a kind of smart way which could be very dumb way as well we don't yeah  Dimitri, I would say you already have an AI tool that you have custom built.  So I must say that you are very much aware about the AI and the advent of AI in this niche.  I would like to add a few things here.  AI is a very good tool if you use it, obviously you know that, but if you talk about content that we are writing and publishing for the sake of convergence and for the sake of Google to actually recognize recognize it as an authority we can't completely rely on ai ai can be used as an assistant but completely relying for example if we select if we use air for the selection of a topic.  We can't make sure that the topic that AI is providing is relevant, right or not, was first thing.  Secondly, the main thing before we actually write a content is devising a proper content brief.  So a proper content outline has to be prepared so that we do know what's the main intent of the user with that query and what our main goal is in order for before we actually write the article. So what our goal is from that content is, but that has to be outlined first.  So if we use AI to create outlines, AI doesn't create very good outlines. So it's our experience and we have been talking with clients in the market.  So AI doesn't create very good outlines.  It has to be a very manual process.  So if you do it manually or with using AI, you can create tens of outlines within two, three hours.  But what process we follow?  It takes us like three, four hours to just create one content outline because the art process is completely manual and we have to actually read the articles that are ranking for that particular key.  So, I mean, We do utilize AI, but we can't completely rely on that.  Thirdly, once the outline is done, content writing is something we can do very good writing using AI, but again, don't properly humanize the content and if the content is not optimized as for the latest Google algorithms, it's not going to work.  Google is very smart.  It actually identifies the content, which is a return.  Not even writers who are actually very much aware of how to use AI tools or GPT or any other tool, they simply read it and they can even identify that it's not a human content or it's an AI written one.  Yeah, I got your point. We don't have that much time, so tell me one thing. Who is actually doing that?  No, no. I have a complete team in my company. So there is an SEO specialist.  I have a team of writers and then editors as well. So the best way to go is to go to the SEO specialist.  go forward with i was just explaining this because uh just to let you know our composite is completely manual and it's not focused on completely doing it automatically so it takes us like three four days to just craft one content piece that we submit. So our main focus is quality, not quantity.
- [12:08] This chunk is part of a conversation between Waqas Arshad and Dmitry Grankin, where Waqas discusses the importance of content marketing and blogging for driving organic traffic to Dmitry's AI product. He emphasizes that consistent content efforts over six months can lead to a natural influx of visitors and enhance the product's visibility in search results for relevant keywords.
  > Waqas Arshad:  So let's say you start working on this on your content marketing and your blogging side and you spend six months on that after six months you will actually start to see an inflow of traffic on the regular basis and it will be a continuous inflow right so you don't need To spend money and then people would be coming onto your website naturally without spending any money, right?  And it will be giving authority to your landing pages as well.  So people who are searching for the keyword, like for example, AI platform or AI meeting assistant or a transcription tool, they will actually see your product ranking overall.  And I completely understand that.

##  [25] - December 02, 2024 17:27

- [07:37] The chunk occurs during a conversation where Dmitriy Grankin is discussing various topics related to a meeting and its transcription features, and it transitions into a brief exchange about a person named Лукаш Гоэш, indicating a shift to a more casual or personal topic amidst the technical discussion.
  > Dmitriy Grankin:  okay What's his name?  Лукаш.  Лукаш Гоэш.  Вот.  Какой-нибудь...  кассу и курицу.

##  [8] - November 25, 2024 13:30

- [00:00] The chunk expresses strong negative emotions and insults directed at an unspecified individual, reflecting frustration or disappointment, and is part of a larger dialogue that continues with similar sentiments.
  > :  Идиот просто какой-то, кретинический.  Кретин просто.

##  [22] - November 25, 2024 20:17

- [00:41] The chunk is part of a conversation involving Dmitriy Grankin discussing the current status of various tasks and processes, indicating a relaxed atmosphere as certain activities and individuals are on hold.
  > Dmitriy Grankin:  появилась никакого как у транскрипта Это нормально. Пока нормально. Мультирасист пока отдыхает.  Мальчик соседа тоже пока отдыхает.  Также прочая макаронная фабрика тоже отдыхает.

##  [20] - December 11, 2024 18:31

- [00:00] This chunk is part of a conversation between Lara Vargas and Dmitriy Grankin, where Lara discusses a recent incident involving windows falling from a building, expressing concern about the construction quality. The dialogue transitions into a discussion about marketing strategies and influencer outreach.
  > Lara Vargas:  the code uh that's what the the fire brigade said maybe the code made the glasses uh change the the you know the the structure and so they fell on the street it was it was a bit scary but thankfully everything is settled now they had to come in and and change everything it's pretty strange that windows just fall out of the building  yes apparently it had already happened to  other people before in this in the building oh yeah yeah okay so so i think it's it's uh the way that it was built it was bad yeah i don't know some mistakes in the construction or yeah yeah anyway that yeah i also had
- [00:38] The chunk occurs early in a conversation between Dmitriy Grankin and Lara Vargas, where they discuss unexpected events in their personal lives before transitioning into a professional discussion about influencer marketing strategies and outreach methods.
  > Dmitriy Grankin:  Today I had unexpected events about kids.  It's okay, sorry about that.  So last week you had asked me to think about ways to make my connection, is my connection good?

##  [26] - December 05, 2024 10:02

- [07:43] This chunk is part of a conversation between Oleg Ageychev and Dmitry Grankin, where Oleg discusses his desire to create a personal assistant that learns from his interactions and the sensitivity of the data involved, particularly regarding copyright and legal issues. The dialogue focuses on the functionality and security of the assistant being developed by Dmitry, which is designed to handle personal information securely.
  > Oleg Ageychev:  все там чему я ее обучил про свой контекст про то как я работаю про то там вот я прямо вообще даже без разговоров с людьми прямо с ней так сел и поговорил много раз плотно хочу себе такого вот ассистента воспитать вот значит наговаривая ему разные там штуки и эти данные все все что я наговариваю для меня она прямо сенситив потому что это там в том числе этом про авторские права и так далее юридические там вопрос вот я Это все хранится у тебя секьюрно.  И я имею в виду, что у тебя же ядро одно.

##  [14] - November 25, 2024 20:50

- [00:13] The chunk is part of a conversation where Dmitriy Grankin discusses the status of a task or project, expressing fatigue and the expectation that it will soon be completed.
  > Dmitriy Grankin:  И, по идее, эта штука должна сейчас у нас закрыться.
- [01:23] The chunk is part of a conversation where Dmitriy Grankin expresses his exhaustion and frustration after a long day, while discussing the status of a task or project that is expected to be completed soon.
  > Dmitriy Grankin:  Короче, в очередной раз умотался, ушатался.
- [02:48] The chunk is part of a conversation where Dmitriy Grankin discusses the status of a task or project, expressing fatigue and the absence of packages, leading to the decision to close the matter.
  > :  Теперь мы её, например, закрываем.

##  [32] - December 18, 2024 08:01

- [10:00] This chunk is part of a discussion among team members about design changes and tasks related to a project, specifically focusing on modifying a table for better clarity and organization of information. It highlights the need for visual adjustments and collaboration between team members, including references to specific tasks assigned to individuals like Vlada and Sandra.
  > :  Давайте еще раз.  Тем не менее, мы отдали уже то, о чем говорили.  Поменять цвет.  Вот там штучки.  Какие-то эти штуки мы отдали, да, Саша?  Для оценки.  мы отдали до то что было дает в лучшем случае у нас еще три с половиной часа есть  есть у саша оценка правильно вот вот это все вместе вряд ли на три с половиной часа нет вот я просто вижу что здесь стоят какие-то комментарии мои в том числе Сейчас личный кабинет, премиум мы скроем, да, ну все, да, премиум мы скроем, ну и че, ну В принципе...  Просто под тарифом, мне кажется, нужно нарисовать им чуть-чуть повыше.  Вот, вот, вот.  Вот такую вот маленькую табличку, чтобы они понимали, потому что я бы...  текста который написан мы получим кашу я бы если честно это нарисовала потому что там очень много комментариев по исправлению и они сто процентов где-то что-то потеряет но супер а нарисуешь у тебя Считывая комментарии, которые на полях есть.  Да, с владенными руками нарисуй всё, что ты хочешь.  Не, не надо владенными руками рисовать табличку.  Это странно.  А, ты хочешь... Подожди, ты хочешь...  Хочешь ее просто как таблицу оформить, тогда окей.  Если ты хочешь прям дизайн...  Нет, жопу дизайн, дизайн тот же.  Просто чтобы ты правильно сказала, чтобы не по верхней строчке таблицы название, там, описание, ну вот для тупых.  Окей.  Нет, ну смотри, вариант, наверное, можно и дизайном передать, как бы, да, если Владе, допустим, давайте так, сегодня Влада допиливает главную страницу, завтра у Влады...  влада не занимается главной странице пока я обсуждаю это все с инвесторами вот например ты можешь в этот момент взять эту задачу как раз вот и передать в дизайне прошу ответишь как бы История с тем, что мы им даем какой-то... скорректируйте текст.  Я говорю, мы здесь постоянно въебываемся в баги.  Потому что они где-то что-то делают руками.  Хорошо.  Мы в дизайне хотим, чтобы...  Влада, что сделала? Вот в этих столбиках текст поменяла или UI изменила? Ну, вообще, визуальный стиль.  Текст поменяла, что-то нет в виду, потому что у нас же есть тексторифы, нам бы хотя бы текст поменять.  Ну, тогда Сандра сделает табличку, отдаст Владе,  а Влада сделает уже для терабита.  Ну, допустим, так тоже можно.  Только единственное, что Влада сделает, делай, пожалуйста, на новой странице создание.

##  [17] - December 20, 2024 13:25

- [01:42] The chunk is part of a conversation between Dmitriy Grankin and Anton Saburov discussing the development and functionality of an AI chat tool that organizes information and assists users by utilizing transcriptions, referencing a specific example of "Клод AI" as a model for their own product.
  > Dmitriy Grankin:  которая накапливается, у тебя получается чат, AI-чат, в котором все очень хорошо доступно, вся эта информация по полочкам.  То есть в качестве примера есть, например, Клод AI.  Там есть проекты, если ты туда аккуратно будешь складывать транскрипты, то ты получишь очень хорошего ассистента.  ...происходит автоматически.  Автоматически, но и там почему-то у них проект лимитирован по размеру.

##  [34] - November 25, 2024 12:54

- [07:14] The chunk occurs during a technical discussion where Dmitriy Grankin is troubleshooting a system or application, expressing frustration about a component not functioning as expected, leading to the statement "Dead."
  > Dmitriy Grankin:  Dead.

##  [5] - December 03, 2024 11:05

- [10:06] This chunk is part of a conversation between Dmitriy Grankin and Daniel discussing the competitive landscape of AI transcription tools, specifically focusing on the limitations of competitors like Fireflies and TLDV, and highlighting the advantages of Dmitriy's solution, which offers more flexibility and privacy for users.
  > Dmitriy Grankin:  That looks like a deliberate decision they made in order to...  abstract themselves from the general market and dig deeper into the corporate sphere okay that's their choice it it's it's been beneficial for us yeah and and it's a choice that doesn't i i don't see okay they  could like have some extra features when you have the google workspace but they are just liberally not leaving you to use it if you are not a google workspace user which is they are removing this market from uh which yeah of course they are like focusing on specific customers and that's fine.  But I think today there are many options from Google Workspace.  Like you have other providers more like less centralized as Google, as Microsoft.  And some people like, I don't like the idea of being stuck to any provider.  So I don't like that.  like the idea and your solution seems more like free. You install your Google Chrome adapter there and you can use it if you don't like it you uninstall it and your data is not being shared with like google or anything anything besides the thing that you chose to use So I like this approach of being less dependent on other existing ecosystems to work.  Yeah, and I-  Depends on Chrome, let's say.  If I use Safari, for example, I wouldn't, which is one limitation, but I think it's just a strategic moment that you are using.  using Chrome as a first implementation.  And if it works probably later, you will create plugins for other browsers.  It's just that.  Yeah, yeah.
- [06:28] This chunk is part of a conversation between Daniel and Dmitriy Grankin discussing the use of AI agents in online meetings, specifically focusing on the differences in their functionalities, privacy concerns, and user experiences with various transcription tools like TLDV and Fireflies. Daniel expresses his dissatisfaction with existing agents and highlights the need for better language identification and usability.
  > Daniel:  participating in talks in meetings online meetings and i would say that almost everyone is um is like using any kind of agent uh one thing that i see that is different on your side is that when one thing that bothers me is that when you have like a meeting with five people and everyone has its own agent the agents joins the meeting and there are like five people and another five engines like in the meeting which is creepy to see like all these agents in some way it's kind of transparent because you see that there's there's an ai transcribing the thing in your case like nobody knows that you are transcribing having the thing and like using AI because you it's it's a plug-in on your own machine right so I can see that you are using it which i don't think it's if it is a good thing or a bad thing but just it's it's different it's a different approach so i assume that you are not using the you are using the real time audio on the machine of the user and not the uploaded audio that the meeting has probably this is how other agents do they they can do work afterwards uh while your agent needs to be like active during the meeting uh like the person needs to have it actually also for the privacy  concern we actually do send a message to the meeting chat informing the participants but it didn't work this time for some reason I need to check why is that but yeah we are trying To comply with regulations we need to do it and we have another one single competitor doing the same thing and they also doing it similar way they they send a message to the to the meeting chat and actually it's a competitive advantage over those who are sending bots because because people tend to hate them.  Yes, yes.  Yeah, I would love to use it for a while, to test it, to know, to like feel, how it goes because i would say that i am not that uh happy with it with the agents that i used so far uh one one agent that i was Usar o TLDV tem um problema que não identifica a língua, então eu preciso dizer manualmente que isso é um encontro português, isso é um encontro inglês.  And this is so bad because it's after the meeting, if it's not in English, it sends an email with a completely crazy, uh, uh, yes.  abstract of what happened and it's so weird yes having and i can't change the language after the meeting happened so it's it's pretty bad so it's not working for me and There is the other Fireflies, I think it's Firefly the name.  Yes.  Yes.  Something like, which is a little bit better dealing with languages, but has some other issues that I didn't like it as well.  Which ones are those?  Yeah, the Firefly, Firefly flies door AI.  Yeah, the fireflies.ai.  Yes.  Let me go into my...  the fireflies you need to have a google uh workspace account oh really allow you to use your ordinary gmail account for recording it so you need to be a company organization and you need to have Google Workspace, which is like also terrible because I don't wanna have Google Workspace to use or Microsoft.  uh whatever account so they they want you can't create an account if you don't have this uh if you don't use google if you don't use microsoft teams or uh anything as long as I know so it's it's not useful for individuals or for like people that don't want to use Google's tools especially people they don't they don't want to have all their stuff in Google or in Microsoft so that's why for Fireflies also doesn't fit my needs.  Interesting.  I didn't know that.

##  [24] - November 26, 2024 15:01

- [17:21] The chunk occurs towards the end of a conversation between Dmitry Grankin and Alisa Molokova, where Dmitry offers Alisa the opportunity to try out his product for free, emphasizing its benefits and expressing gratitude for their meeting.
  > Dmitry Grankin:  Вот, и предлагаю попробовать продукцию поиспользовать.  Это пока бесплатно, поэтому пользуйся.  Единственное, что сейчас я его в ваш сайт подниму обратно.  вот вот ровно такие в тех обстоятельствах которые сейчас находимся вот он несет  несет пользу в процессе и после хорошо договорились спасибо тогда рад знакомству и

##  [1] - December 17, 2024 05:31

- [03:20] This chunk is part of a meeting discussion where Dianna Lloyd introduces herself and her background in marketing, CRM, and HubSpot, highlighting her experience and willingness to collaborate on the HubSpot platform with Shannon and Samantha.
  > Dianna Lloyd:  freak. So this is what kind of got me into this because I really owned HubSpot in the last enterprise I was at with five different brands on the HubSpot and an implementation and all of that.  Um, and I've been doing CRM in campaign management and email marketing for so many years.  I just, um, I'm in my happy place here.  So that's just a bit about me so that you can, you understand sort of, I guess the, where I'm coming from and, and all of that. Um, and I've also have a design background. You wouldn't know it from these emails that I put together, but, um, I have a design background, so I'm also happy to talk about those types of things with you guys as I sort of get to know you and work with you on this.  of your HubSpot platform.  Brilliant.
- [07:49] This chunk occurs during a meeting where Dianna Lloyd is discussing the initial steps taken to review a project brief related to email workflows and marketing strategies, following introductions and a recap of the meeting's purpose.
  > Dianna Lloyd:  thank you everybody's ready there it goes okay um recording will start soon i think it's recording cool okay what was i so anyway so yes so the first thing that i looked at when i was um i had the the original brief and this was the brief that I received.  Yeah, this is all of it.
- [02:44] This chunk is part of a conversation during a meeting where Dianna Lloyd introduces herself and her background in marketing operations, CRM administration, and HubSpot, highlighting her role in supporting Marcus in sales and her enthusiasm for learning more about the platform.
  > Dianna Lloyd:  the years, but suffice to say that on top of, you know, marketing operations and CRM administration and, you know, really being the owner of my CRM and HubSpot Um, I've got a, uh, an extensive experience just in the general marketing activities and then sales enablement activities that lead into the customer, um, sort of livelihoods Life cycle activities.  So that's where I come from.  So Marcus has brought me on, I guess, you know, to be the end to his, his sales yang in a way.  And while I'm familiar with all the sales side of everything in HubSpot, this is a good opportunity for me to learn.  And I'm just a massive HubSpot fan.
- [14:25] The chunk is part of a discussion during a meeting between Dianna Lloyd, Shannon Clegg, and Samantha about optimizing their marketing processes using HubSpot. Specifically, Dianna emphasizes the importance of collecting website URLs from leads to enhance data insights and reduce spam, contributing to the overall strategy of improving customer engagement and marketing effectiveness.
  > Dianna Lloyd:  Website URL is another thing I wanted to talk about making that required and there's multiple reasons for it.  of them is that you can get more AI data on a company's information if you have the domain.  So if we get their company, their website URL, you can start utilizing some aspects of HubSpot that will give you some insights into the company if it can crawl that company URL.  So I would- That's great.  That also helped with spam.  In spam.  Yeah.  Oh, yeah.  Brilliant.
- [00:00] This chunk is an introductory conversation between Dianna Lloyd and Shannon Clegg, where Dianna shares her background and recent joining of Vera Digital, while also inquiring about the meeting's participants and discussing the upcoming Christmas break. It sets the stage for a collaborative discussion on marketing strategies and workflows related to their work.
  > :  Hi.  Shannon, sorry I jumped out of one meeting and into another.  I haven't even had a chance to put my hair tie on.  That's okay. I'm well, thank you. And yourself?  i'm good hot are you in you're in sydney right yeah it's so warm today miserable my daughter's at the beach and i'm extremely jealous and i don't  That's okay, is Sam joining us or is it just us?  I've only got your acceptance of the meeting.  Okay, that's fine.  knock it out together absolutely absolutely are you all ready for christmas break and all of that or yes checking out on friday so very excited um okay let me just i'm gonna share my screen perfect that would be great thank you  so i'll just give you a little bit about me  So, um, so basically I've just joined Vera digital.  So I don't know if you've worked, you haven't worked with Marcus before, have you?  Not particularly, I've had a, I've been on a couple of emails with him, but yeah, no.  Actually, Samantha is here.  Oh, she's here.  No, sorry, computer's not working, so I'm on my phone.  No problem.  That's fine.  You won't see my pretty pictures, but that's all right.  Oh, I should, I should be.  oh you can see okay okay yeah yeah um yeah so i was just saying that i've uh just sort of introducing myself so um i just started with vero not too Long ago, my background is in everything marketing, B2B, B2C, lots of acquisition and product marketing, direct response, marketing operations, and brand management and campaign management and all of the things that go with that and being probably burned out by all the things I've had to do throughout.
- [06:10] This chunk occurs during a meeting where Dianna Lloyd discusses her initial observations and the creation of a spreadsheet to understand the customer journey through various workflows in HubSpot, specifically focusing on the initial inquiry flow. The conversation also includes a request to record the meeting for a colleague's reference.
  > Dianna Lloyd:  So that's what I kind of put together a little spreadsheet and it's not fancy, but it's It just is a helpful way of just trying to understand that, that journey, you know, as they come through.  So, um, when I first started building mediums, I wasn't really aware of entry to the whole entire, you know, setting them off into a welcoming email workflow and all of that. So I went back and I just started looking at what the different workflows in And so the first workflow that I saw is actually this workflow here, which is the initial inquiry flow.  Yep.  Sorry, Diana.  Is there any way we can record this?  yeah just so that i can um send it to our colleague georgie who'll probably want to be across this as well okay i love that you always remember that sam  okay i i'm not i'm used to working in teams so recording is do do do i think if you press the  Yeah, me too.  I see more options. Oh, here we go.  Manage recording. Here we go.  Got it.  Do you want a transcript too?  whatever whatever's there yeah that's fine i'll just start recording okay
- [08:11] This chunk occurs during a meeting where Dianna Lloyd discusses the process of building email marketing campaigns (EDMs) and the initial inquiry workflow in HubSpot. She seeks clarification on how leads enter the system through contact forms on the website and outlines the subsequent steps taken to manage these leads, including task assignments and automated email responses.
  > Dianna Lloyd:  Then I started building the EDMs and had some questions so i went back and i looked at how do they how do they get into this whole this whole journey right and it starts out with this uh workflow which is the vd sales initial inquiry flow and just please correct me if I've got something wrong.  It's just this is this is what I found.  And if I have it wrong, please let me know.  So is this is this that workflow just from people entering an email on the website is that correct i believe so so it feels  fill out any contact us form inquiry right yeah so if i go to the mark I think it's anything on this website is what that's talking about.  And Samantha, by the way, I need to talk to you.  about this form here because it's different oh yes my um my colleague georgie  and our web developer who put it there are sorting that out for you  okay because you'll want to get all that information right yeah so I pass that I pass on  your message to them and they're just working on it so much easier okay so if I fill out a form where's  a good place to fill out form.  I'll go to contact us.  So this form here or any form that's similar in shape and size to this form fills out a contact in, um, with all the information into the HubSpot portal right so i believe this is a HubSpot form now so they fill out this form and they come in and samantha gets made the contact owner they're set to a new lead status they get a life cycle stage of lead and then there's a task created for samantha that gives them all the information um and they're set as a marketing contact and um They get this email, and this email, I'll just show it here, is basically, thanks for reaching out, here's our marketing checklist.  and we're excited to learn more about you and the expectation in that is that Samantha has some time to contact this person mm-hmm because you've been given that task.  Do I have the story right so far, Samantha?  Oh, she's gone.  She'll be back.  She'll be back, okay.  to you sorry I went to unmute my mic and I exited but yes that is correct all the time okay
- [19:00] This chunk occurs during a discussion between Dianna Lloyd and Sam regarding the timing of automated email follow-ups in a marketing workflow. They are deliberating on the optimal delay for sending emails to new leads after they fill out a contact form, considering factors like response speed and recipient engagement.
  > Dianna Lloyd:  So manual enrollment, and I've set it to delay for a day to give you time contact them i will delay it till monday tuesday wednesday or something like that i won't have it so that they get an email on the weekend if if you would prefer um  Sam, your thoughts on that? I don't mind if it arrives on a weekend.  Yeah, I would just, I think speed is the competitive edge, especially now. I agree.  so i would just delay for a day well the reason being is because they've just received as i said  they've just received this email yeah and then a lot too even in one day, you know.  Okay, so just a bit over 24 hours then.  So not the next day, but the day after.  Whenever they've done it, if they filled out the form  At 10 p.m., it'll be 24 hours later.  Okay, and that's fine?  Is that what you're recommending?  I do.  I recommend, some people like to wait until 10 a.m.  to send these, but what, um, you know, there's a lot of thought that people are on, on, they're on their phones, they're on their computers.  at the same times when they have that time to sit and play around with these types of things if you send it at 10 a.m you're most likely to hit people when they're in the middle of their workday yeah which isn't necessarily about  But if they are, I think you can assess how quickly people are opening because you're going to have that data.  So if I said it at two o'clock,  10 a.m. um then what could happen is they fill it out at 10 p.m. and it's 10 a.m. the next next day so it could be a day and then 10 a.m. or it be wait i don't know 12 hours it could be wait till 10 a.m the following business day so i i need my my suggestion would be to say just delay it for a day and have a look and see what happens agree and then we can do okay let's do that and do some a b testing so if they if they if they sent it at 4 30 then they're going to get the email at 4 30.
- [17:48] This chunk is part of a discussion between Dianna Lloyd, Shannon Clegg, and Samantha regarding the setup and management of marketing workflows in HubSpot. Dianna explains the importance of the initial workflow in establishing contacts and outlines the structure of subsequent workflows, emphasizing the need for potential future adjustments to better cater to different customer journeys.
  > Dianna Lloyd:  Um, okay, cool. So, where was I? Yes, so I finished talking about this. So, um this information this web this workflow basically sets them up as a good contact with the information that they need their marketing contacts so there's a lot of back-end processes that we need to be able to market to them and to give you some data and to set them on your workflow and your pipelines and all of that for creating them and making them a customer and creating a deal and all of that and it has the one welcome mail, which I just wanted to go.  That was the most important thing I wanted to point out.  So, um, the next workflow, then the way I've built it is that this workflow will be actually triggered out of this workflow so i'll create an extra little thing that comes out of here an extra action and it says go to this workflow yep so this one's doesn't have an entry point, it's completely rely on this one to send them on.  So in essence, it's a continuation of the same workflow.  Um, but later on, We might make some decisions to split them out and not do that, which is a later thing that I wanted to talk to you about, which is why I didn't say just continue the whole thing.  It's better to split them out if it's now this different nurture journey.  because we may have multiple things that come off the back of this in future.
- [10:51] This chunk occurs during a discussion between Dianna Lloyd and Shannon Clegg about the workflow for responding to inquiries received through a contact form. Dianna inquires about the typical response time for reaching out to new leads, and Shannon provides insights into their promptness in communication, while also considering future scalability for international markets.
  > Dianna Lloyd:  um yeah so and when you get this notification in the workflow I just have a quick question for you about that one of my questions was is how long does it take you usually to try  reach out to that person uh like maximum two hours we're very fast yeah the only Any time it's long, like a long wait time is if it comes at like 4 a.m. or 3 a.m. and we're sleeping, but it'll be the first time.  Yeah.  Even if it's Friday night, I'll still respond.  and I see it, or I respond over the weekend, that's more than fine because I'm that person that checks my emails over the weekend, but If it's overnight, then no, it's the next morning.  If we're gonna future-proof this process moving forward, going into North America, the Middle East and Asia, it's probably that lead time will be longer. Yeah, of course. I mean, those that come in overnight are usually international.
- [21:56] This chunk is part of a discussion between Dianna Lloyd and Sam regarding the content and strategy for a welcome email in a marketing workflow for Taurus. They are evaluating the messaging, specifically the inclusion of a checklist and the relevance of the Taurus brand versus Taurus Shop, ensuring clarity in communication for potential leads entering the workflow.
  > Dianna Lloyd:  because that's when there's that's when they're commuting that's when they're doing stuff and they might go oh yeah right and they might have a look okay so sounds good your first email is a welcome and a bit about taurus noble The CTA, or the action that we've said, again, is that checklist, which is which is repetitive so I'll just it's not meant to shout in red but it's just so that I remember so um so to look at the email I've changed some of the wording to be, if you haven't already, be sure to download our free resource on Taurus Marketing Checklist. However,  Sam, is this a good point, is this a good email for us to embed the bullseye methodology and then link it through to the website?  Yeah, absolutely.  I think also the other thing to be careful of is Taurus isn't our bread and butter it's not it's not the only thing we do like we've been around for 29 years because people come to and really shop is for that Northern Hemisphere audience.  So it's not just about shop.  So is this when people buy from shop or is it just he's been there inquiring because he says welcome to Taurus shop I can answer  that the contact us trigger to bring people into this workflow has nothing to do a shop perfect okay so that's a good thing we've just uncovered because
- [15:02] This chunk occurs during a discussion about optimizing a contact form in HubSpot, specifically addressing the inclusion of email domains and website URLs, as well as potential data validation features to enhance lead capture and management processes.
  > Dianna Lloyd:  Is there, like we can say no Gmail addresses, things like that.  you know how it's been set up but um I can have a look at that and get back to you if I have any suggestions on the form other than that it's okay if some of them I know most  spams our Gmail or something like that.  But we did recently get a client who still uses a Gmail account.  So I wouldn't completely abort it.  I know it's quite a rare case, but he does.  And you might actually, now that I'm thinking about it, you might get URLs.  People don't have a URL because it's personal branding.  Is that possible?  That's possible.  Sometimes, yeah, that's possible.  And then the other thing is they don't have a URL because they're trying to build a new website or they're trying to launch.  i mean it's still maybe we just have it um it's there now so if they if they enter that's great  it just we can't make it if you're gonna have clients that don't if it's not completely b2b actually make it that yeah yeah okay so going back to this um sorry sorry just an afterthought  we could make it We could make it compulsory and if they don't have a website, I don't know if we can put an put in if you don't have a website, put in your LinkedIn profile.  You know, I don't think that can be done, but I'll have a look.  I look, I've built quite a number of forms on HubSpot and I've never been able to see anything that's not.  Yeah, that's fine.  There's some new stuff though with their forms.  There is some conditional data validation.  So let me have a look, okay?  Okay.  that's a good idea it was just an afterthought data validation and i don't know enough about it  You know more than I do, that's for sure.
- [24:00] This chunk occurs towards the end of a meeting discussion where Shannon Clegg and Dianna Lloyd are reviewing and clarifying the content and workflow of automated emails related to Taurus marketing. They are specifically addressing the welcome email and checklist, discussing the need for a review of the email terminology and ensuring the correct messaging is in place.
  > Shannon Clegg:  terminology sorry it's um those are the emails that i put together sam oh yeah i I don't recall doing the first one for the welcome to Taurus.  Was that added?  With the checklist?  Yeah.  Probably prior to my time.  Right.  So that's why it's worth I go in and just read  review each one of them but i just don't know where to go in from uh i think you just go in  from those links that diana had sent previously in the email i'll share this  document with you if you guys do you want it now yeah okay yeah that's good and then i can just  check them overnight for you yeah and i think now that we have the context it's a lot easier  to understand the flow
- [04:22] The chunk is part of a meeting discussion among team members, including Shannon Clegg, Dianna Lloyd, and Samantha, focused on improving digital marketing strategies for Taurus marketing and Taurus shops. Shannon outlines the team's findings regarding underutilization of their database and introduces a new email sequence designed to guide potential customers through their purchasing journey.
  > Shannon Clegg:  So just to put a few things into place, I've been working with, Sam and the Taurus team for about five months, working on all of their digital marketing and SEO components for Taurus marketing, but also Taurus shops.  So a part of our findings and discussions throughout those months were that people weren't, we weren't utilising the database ...to take people on that journey of tourist shops.  So we've come up with a sequence of emails to help people along...

##  [13] - December 03, 2024 14:31

- [00:16] The chunk is part of a conversation between Aleksandr Lavrinenko and Dmitry Grankin, discussing technical experiments related to virtual microphones and audio processing tools, specifically focusing on the capabilities of the Crisp software and its applications in their respective projects.
  > Aleksandr Lavrinenko:  как делаю что-то похожее сейчас для эксперимента на камеру Крисп. Крисп тебе позволяет создать виртуальные штуки.  И потом он по-разному с этим работает.  Виртуальные что? Что виртуальные?  Микрофон динамики, он означает...  убирал шум оттуда лишний чтобы звук был чистый а потом они стали еще одним ноутекером на стероидах я немножко вдохновляюсь сейчас с маленьким экспериментом Возможно, ты знаешь больше, потому что ты все-таки разработчик, условно, ноутейкера уже довольно давно.  Ну, тем не менее, я очень мало взаимодействовал с продуктами конкурентов.  Просто руки до этого не доходят.  Ну, бывает такое.  Бывает.
- [07:15] The chunk is part of a conversation between Dmitry Grankin and Aleksandr Lavrinenko discussing technical challenges and experiences related to speaker diarization and identification in audio processing. They explore the complexities of developing effective models for separating and identifying speakers in audio streams, highlighting the lack of existing solutions and the difficulties encountered in their respective projects.
  > Dmitry Grankin:  если я тебе хочу предложить потерять бесперспективно полгода я тебя предложу заниматься идеализацией я сейчас Как диализацию делать, потому что я сломал диализацию, знаешь, много зубов, вот, и это серьезный научный наследник.  Но это просто надо делать нормальную модель с нуля, ее просто нет.  То есть нет нормальной модели, потому что диоризация, потому что диоризация это же по сути отделение спикеров и кусков.  чтобы ты понимал кто о чем говорит и ты говоришь там что нормально сделано такой тузы нету готовы нужно прямо к самому собирать что-то ну  У меня задача была speaker identification. Я делал из этого алгоритма модернизации, я вытаскивал бидинги что сама по себе уже потребовала переработки библиотек которые предлагались, потому что имбейтинги нет.  Потом всякие операции с имбейтингами для того, чтобы вычислить, получить имбейтинг спикера.  Тоже ничего не тревожит, потому что эти беденки не какая-то фиксированная величина, они у тебя вот так вот, да, и тебе нужно их кластеризацию какую-то проводить.  это капец как было интересно но устойчивого результата я не получил окей тогда я думаю что
- [00:00] The chunk is part of a conversation between Aleksandr Lavrinenko and Dmitry Grankin discussing technical experiments related to virtual microphones and audio processing, as well as updates on their respective products and projects in the field of online consultations and transcription technology.
  > Aleksandr Lavrinenko:  Спасибо, что рассказал. Как раз мои технические эксперименты оставили мне палочку в колесах.  с каким-то там микрофонным мостом но у меня виртуальный микрофон динамики
- [05:44] The chunk is part of a conversation between Dmitry Grankin and Aleksandr Lavrinenko discussing audio streaming and real-time transcription technologies, specifically focusing on the technical aspects of a Chrome extension that enables multilingual transcription and speaker identification.
  > Dmitry Grankin:  это стримит аудио я делал real-time транскрибацию ну как real-time задержка секунд 12 получается вот Но зато возникло еще одно конкурентное преимущество, это то, что мультиязычность, адаптивная мультиязычность.  переключается определяет переключается на язык в моменте а как ты потаскиваешь но  если прям технически уходи сюда как ты подключился к аудио потоку из google extension а то есть ты получается у тебя есть микрофон и колонка условно ты к ним подрубаешься ты как бы становишься как буфер между ними как-то сбоку подсасываешься я не но это chrome extension может получить такую это базов я просто секс точными хрома очень мало работал я там вообще не эксперт то есть ты можешь с пользой chrome extension получить доступ к микрофону без всяких штук до того же что микрофон  ...фону с разрешения пользователя получить, и есть разные способы получения, ну, не системного звука, а того, что в хроме происходит.  И дальше ты, получается, режешь это на маленькие аудиодорожки и все это транскрибируешь по отдельности, и вот у тебя диализация или какой там алгоритм примерно?  Диализация – это…
- [19:13] The chunk is part of a conversation between Aleksandr Lavrinenko and Dmitry Grankin discussing the development and features of their respective software products, including virtual microphones, updates on their dashboard, and the use of AI models like GPT-4 mini for transcription and processing.
  > Aleksandr Lavrinenko:  You know, userful notes. Виртуальные микрофоны, их использование, протообновление новостей, стерлинги поисков с психотерапевтами, детали. Неплохо, кстати.  Прикольно, виртуальные штуки.  Прикольно, что он вытянул виртуальные штуки.  Отличается новым дашбордом.  интересно интересно посмотреть как у меня там промкинг работает я видимо там на чинов и  Thoughts подключал, потому что в дешборде я пока не занимался промпт-инжинирингом, он там такой пока слабенький.  Угу.  А ты из моделей на ком остановился? Это уже, скорее, спортивный интерес.  У меня все работает на GPT-4 mini.  34 а менее понятно ну это просто как бы это просто самая дешевая модель которая рабочая лошадка для всего а то есть и не только беда получается чем-то

##  [10] - December 02, 2024 15:30

- [00:00] The chunk is part of a conversation between Dmitriy Grankin and Igor Lyubimov, where they discuss their recent activities, specifically focusing on coding and work-related tasks.
  > :  Да, Игорь, привет.  Что, как дела?  Ничего, вот сидел несколько недель, кодил.
- [10:11] The chunk is part of a conversation between Igor Lyubimov and Dmitriy Grankin discussing the challenges of managing client calls, particularly the difficulty of remembering details after the calls and the reliance on transcription tools to review discussions.
  > Igor Lyubimov:  У меня еще клиентские такие звонки, они все, ну, типа, однотипные.  То есть там мы частным там скинем эту ссылку, скинем ту ссылку.  ссылку. Типа вот зарегистрируемся или мы там подумаем, через неделю вернемся.  Вот. И все.  А я на следующий день уже забываю все, что мы там обсуждали.  И вот поэтому я иду в тактик и возвращаюсь, и пытаюсь прочитать.  Транскрипт.  Да, прочитая транскрипт, он не всегда попадает, а еще вот когда язык не переключается.
- [26:23] The chunk is part of a conversation between Dmitriy Grankin and Igor Lyubimov, where they discuss various tools and strategies for managing sales calls and transcriptions. The chunk represents the closing remarks of their discussion, expressing gratitude and hope for future assistance.
  > :  Вот. Ладно, спасибо тебе большое. Рад был тебя слышать.  Да, спасибо за ту. Надеюсь, он мне поможет. Вот тогда на связи. Пока.
- [11:05] The chunk is part of a conversation between Igor Lyubimov and Dmitriy Grankin discussing the challenges and frustrations of using a transcription tool for calls, specifically focusing on issues with lost calls, licensing costs, and dissatisfaction with the service.
  > Igor Lyubimov:  короче это да да да потерянный звонок и мне остается только надеяться что я правильно запомнил или клиенты про меня не забудут Короче, это стресс, это неудобно, и я второй раз сегодня уже писал и говорю, что «ребят, я с вас собираюсь уходить, сделайте эту штуку».  вот так и ну ты соответственно платишь и и это какая-то у тебя сколько лицензий  А я плачу... Ну, сейчас одну лицензию, сейчас там еще один человек появится, буду две платить. Ну, вот, я не помню, сколько...  Сколько она стоила? По-моему, она стоила типа там 12 или 15 баксов. Я потом пошел отменять, потому что был недовольный, что опять проебал какие-то замки. Они мне говорят, типа, переключай Приключительно на 6 баксов. Я думаю, 6 баксов сейчас плачу, потом они меня, видимо, обратно, то есть там, скорее всего, они дали типа на 3 месяца, вот, наверное, через месяц, может, через 2.

##  [33] - November 19, 2024 18:20

- [00:00] The document appears to capture a brief dialogue or exchange, where one speaker is inquiring about a situation or event, indicating a moment of confusion or curiosity.
  > :  Что происходит?

##  [36] - November 19, 2024 09:38

- [00:00] The document appears to be a conversation or interaction involving testing a system's functionality, with the chunk expressing a specific action related to that testing.
  > :  Тестирую работу системы...

##  [29] - November 25, 2024 11:47

- [28:45] The chunk appears towards the end of a conversation where Dmitriy Grankin is discussing technical issues related to speech processing and database interactions. It is interjected by a frustrated response, likely from another person, expressing a desire to be left alone amidst the technical discussion.
  > :  отстань отстань отстань от меня  не хочу ничего не сделала идти заниматься своими делами что ты хочешь  от меня поставь меня в покое пожалуйста закрой дверь  Сокрой дверь, пожалуйста.  Ничего, все хорошо.  Оставь меня в покое, пожалуйста.

##  [19] - November 25, 2024 19:45

- [00:00] The chunk is part of a conversation by Dmitriy Grankin, where he is testing a system or application, expressing frustration about its stability, and engaging in a casual dialogue with others about the status and functionality of the system.
  > Dmitriy Grankin:  Тест, тест, тест, тест, тест.  С. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС. ТС Так, что ж отвалится все-таки, да?
- [00:40] The chunk is part of a conversation by Dmitriy Grankin, discussing the status of a project or system, expressing surprise that something is not failing or falling apart, and reflecting on the situation in a casual manner.
  > Dmitriy Grankin:  И не валится.  Ну вот это что, всё-таки был такой...
- [02:00] The chunk is part of a conversation by Dmitriy Grankin, discussing the status of a project or system, expressing curiosity about its functionality, and indicating a desire to investigate further.
  > Dmitriy Grankin:  Не падает. Пойдем посмотрим, что делает вот этот вот.  Как у нас выглядит с тверочком.  Ну ладно...

##  [9] - December 05, 2024 19:00

- [00:05] The chunk occurs at the beginning of a conversation between Lara Vargas and Dmitry Grankin, where they discuss their current locations, specifically mentioning Lisbon and the cold weather, setting a casual tone before transitioning into more detailed discussions about business and personal interests.
  > Lara Vargas:  I'm on my iPad, so I just want to check on Lisbon.  Yeah, I am. Are you as well?  Yes, on Belas Clube do Campo, not far from Lisbon.  Oh, okay, nice. It's been very cold, isn't it?
- [00:43] The chunk occurs during a conversation between Dmitry Grankin and Lara Vargas, where they discuss their locations, the weather, and personal interests. Dmitry shares his research on obtaining Brazilian permanent residence, indicating a deeper interest in Brazil, which aligns with Lara's Brazilian background.
  > Dmitry Grankin:  Actually, I've been now researching on how to obtain Brazilian permanent residence, believe me or not.  Oh, really?
- [01:09] The chunk occurs during a conversation between Dmitry Grankin and Lara Vargas, where they discuss travel, citizenship, and the process of obtaining Brazilian permanent residence, particularly in relation to Dmitry's interest in collecting travel documents as a Ukrainian-born Russian citizen.
  > Dmitry Grankin:  are you planning on going there to um actually uh i like um collecting travel documents like For a Ukrainian-born Russian citizen, it becomes a necessity to collect different documents.  a portuguese child and so okay you technically obtain the brazilian citizenship quickly so it feels like an interesting interesting game
- [00:34] The chunk occurs during a conversation between Lara Vargas and Dmitry Grankin, where they discuss their locations and experiences with the cold weather in Portugal, highlighting Lara's Brazilian background and her unfamiliarity with the colder climate.
  > Lara Vargas:  for a brazilian like me where uh what part of brazil are you from i'm from rio so yeah i'm not used to this I think the greys is already a lot for me, I mean.
- [20:45] This chunk occurs during a conversation between Lara Vargas and Dmitry Grankin, where they discuss the challenges of setting up a business account on Facebook. Lara shares her experience with the difficulties she faced while establishing her account and mentions her transition from a full-time position at a B2B SaaS company called Build Tricks to seeking freelance opportunities, indicating her availability to assist Dmitry.
  > Lara Vargas:  it is really hard to set up a business account on facebook i also had the same problem when i set up my first one it was really hard to get it back it took me probably like two months back and forth with the the facebook team i managed to get it back in the end but yeah i agree it's very hard they make it really hard but i can i can help you with that setting it up and uh i can give you the admin access after I set it up and change it with you what's up what you've been up to what are you doing in in terms of what like um so i actually am leaving build tricks now what i'm leaving build tricks which is where i i was full-time what what's the what is the company yes it's a it was a b2b sas and i'm leaving So I'm mostly looking for freelance yeah so I'm free to I will be free to help you most of the time as you need.  what you've been doing in that company

##  [12] - December 05, 2024 13:59

- [26:22] This chunk occurs at the end of a conversation between Sunny Wyrzykowski, Nora Köppling, and Dmitry Grankin, where they discuss Dmitry's product and potential consulting services. It signifies the conclusion of the meeting, with expressions of gratitude and a positive farewell.
  > :  great okay pleasure to meet you dimitri thanks guys thank you
- [00:20] This chunk occurs at the beginning of a conversation between Dmitry Grankin and Sunny Wyrzykowski, where they introduce themselves, discuss their locations, and agree to record the call for note-taking purposes before delving into business discussions.
  > Dmitry Grankin:  I'm in Lisbon, Portugal.  Lisbon, sweet.  I just got...  What about you?  I am in San Francisco, the Bay Area.  okay and i actually just got back from uh three months in brazil so i picked up a little portuguese but that's a little different from yours i assume a little same  But we've got a lot of Brazilians here who continue to be Brazilian, no matter.  Cool. So before we start, is it okay if I record?  this call so i can summarize the key action steps for my notes and so no information gets lost yeah absolutely same same for me cool yeah no worries
- [00:00] This chunk is the opening dialogue of a business meeting where Sunny introduces herself and Nora to Dmitry, inquiring about their locations before discussing the agenda of the call.
  > :  Hi, hi guys.  Notice.  how are you good how are you doing pretty good so my name is sunny joining here is nora i'm not sure if you guys already met um but where are you where are you calling in from today

##  [6] - December 03, 2024 13:01

- [13:07] This chunk occurs towards the end of a conversation between Anna Pavlova and Dmitry Grankin, where they discuss scheduling a meeting and sharing contact information after a series of networking events related to a business club in Lisbon.
  > Anna Pavlova (Lupenko):  там может быть в районе 12 сейчас может быть если не пугает вот либо можно в течение дня или ближе может быть к вечеру если вы собираетесь лиссабон по каким-то делам да можно просто попробовать попробовать пересечься вот ну либо я могу пригласить на пятницу как раз на наш завтрак в качестве плюс один у нас есть такое условие что что резидент может пригласить плюс один одного человека вот заодно после этой встречи можете попить кофе с нашими сил в пятницу а во сколько здесь С 10 до 12 это Амарейрош, если на машине, то очень удобно.  Конус Гринстон. Ну, я скину информацию, вот.  Угу.  а я добавлю так вроде да у меня ничего нету хорошо тогда  Тогда напишите мне, пожалуйста, свой Телеграм, я напишу, скину туда информацию по встрече, вот, и на почту тоже кину приглашение на мероприятие, вот.  Вот, а там же кину потом в телеграм-контакт Александра, чтобы вы друг друга нашли.  Я, может быть, тоже буду, постараюсь быть.  Ну, ладно.  Ну, класс, рад был познакомиться. Давайте так.  Да, взаимно. Да, спасибо. До скорой встречи. До свидания.  Куда кинуть, куда кинуть? Я, может быть, прямо здесь сюда кину, чтобы?  да можно или в линкедин где мы общались целом тоже можно хорошо такой  такое количество сообщений после веб-саммита  неразобранных, что я подозреваю,  что у вас еще хуже.  Это точно. Но  Но приходится отвечать в рамках своих обязанностей.  Да, потому что я после веб-сайта ушел кодить, и там вообще прям совсем...  прям совсем ну ладно а я тогда сейчас кинул телеграм-канал договорились я сразу отвечу с
- [11:20] The chunk occurs during a conversation between Anna Pavlova and Dmitry Grankin, where they discuss a mutual acquaintance, Alex Pavlov, who is working on launching 12 startups in the next year. They are coordinating a meeting to discuss this further, with Anna mentioning her availability and the logistics of meeting in Lisbon.
  > Anna Pavlova (Lupenko):  Он сейчас, у него очень классный челлендж за ближайшие 12 месяцев построить 12 стартапов, запустить очень быстро.  Я думаю, что будет что обсудить.  На этой неделе готовы встретиться?  Как его зовут?  Алекс Павлов.  Александр Павлов.  нет не знаю супруг  да супруг  семейный бизнес буду рад познакомиться  хорошо а на этой неделе  где-нибудь в первой половине дня?  Свободно?  Я отвожу детей в школу в Оэйрш  к 8.30.  Угу.  Где вам удобно встречаться?  Желательно в Лиссабоне.  Потом из Эйрыша в Лиссабон очень весело ехать.  Да, особенно сейчас, сегодня.  застовка в метро, и, в общем, все вылезли на улицу.  Окей, а какие  по времени варианты?  Может быть, вы бываете где-то в районе Лыреша-Кашкайша с утра?  С утра, к сожалению, нет. По выходным, только если мы там бываем, может быть, в районе.
- [10:50] The chunk is part of a conversation between Dmitry Grankin and Anna Pavlova, where Anna discusses the membership fee for her business club in Lisbon, the process of connecting interested individuals with their SEO, and the relevance of AI technology in their discussions.
  > Anna Pavlova (Lupenko):  У нас есть членский внос, это 200 евро в месяц, оплата по квартально.  Если это не смущает, то я бы вас познакомила с нашим SEO.  У нас принято с теми, кто интересуется клубом, устраивать кофе с нашим SEO.  Он тех...  тех, ну, специалист по теху, по АИ, я думаю, что вам будет шло обсудить. Вот, поэтому я как раз и спрашивала там, что GPD, или как это работает. Вот.

##  [3] - December 20, 2024 18:01

- [19:47] This chunk occurs towards the end of a conversation between Dmitriy Grankin and Windy Hell, where they discuss the next steps for collaborating with bloggers to promote Dmitriy's product. They agree on a timeline for identifying and engaging bloggers for advertising, indicating a plan to move forward with their marketing strategy.
  > Dmitriy Grankin:  Окей, на чем остановимся?  Я думаю, в пределах двух недель смогу первых блогеров прислать.  с кем можно запустить рекламу.  Ну, окей.  И что тогда, когда в следующий раз  Какой следующий шаг?  Но после того, как на это блогеров, блогер под них.  Ну и всё.  Окей, договорились.
- [07:52] The chunk is part of a conversation between Dmitriy Grankin and Windy Hell, where they discuss their experiences and strategies in influencer marketing, specifically focusing on Windy's recent work with a cryptocurrency exchange and how he collaborates with bloggers to attract traders.
  > Windy Hell:  В последнее время я для криптобиржи Гейтео, партнерской программы для блогеров.  занимался, привлекал антрейдеров.  Как это выглядит?  Как твоя работа построена?
- [13:15] The chunk occurs during a conversation between Dmitriy Grankin and Windy Hell, where they discuss collaboration with bloggers for marketing purposes. Windy mentions the need for an agency contract to formalize the relationship with bloggers, while Dmitriy inquires about the existence of a template for such a contract, reflecting on their previous experiences with unpaid collaborations.
  > Windy Hell:  Ну, агентский договор на работу с блогерами.  Который компания будет подписывать с блогерами.  Ну да.  Окей. У тебя шаблон есть, наверное, какой-то?  Да.  Я не подписывал никакого договора с ним.  Потому что у нас никаких денежных отношений не было.  Собственно, он все сделал бесплатно.
- [12:08] This chunk occurs during a conversation between Dmitriy Grankin and Windy Hell, where they discuss potential collaboration on influencer marketing strategies. Windy suggests exploring various niches and themes for engaging with bloggers, indicating a focus on targeting specific audiences such as IT professionals and designers. The dialogue reflects their brainstorming process and the need for further information to initiate their partnership.
  > Windy Hell:  Ну, я бы попробовал разные тематики.  Я почав без професійних блогерів.  Павловским нишам, все потом айтишников, дизайнеров, такие нишевые облаки.  дизайнеров слушай ну давай так я не знаю тебе какой-то по ресерчить надо или или что Если предварительно окей, давай. Что тебе от меня нужно? Тебе надо подумать, что ты будешь делать, или тебе надо собрать от меня...  ...информацию сейчас. Что тебе от меня надо, чтобы начать?
- [18:12] This chunk is part of a conversation between Dmitriy Grankin and Windy Hell, where Dmitriy explains the technical aspects of his product, specifically focusing on the audio-to-text conversion capabilities using Whisper and the integration with GPT-4 for processing and indexing data.
  > Dmitriy Grankin:  Потому что я могу на любом языке разговаривать, и он моментально переключится.  А под капотом Charger 5?  Charger 5, он же  Не делает из аудио в текст.  Это идет под капотом Whisper.  Whisper это моделька, которая делает из аудио в текст.  И у них там есть несколько размеров моделей, несколько разных версий, которые очень по-разному место занимают в памяти.  Усилительное устройство и так далее. Я запустил самую тяжелую модель. Она очень легко переключается. Потом дальше это обрабатывается GPT-шкой.  И чат тоже на GPT-4 работает.  Нужно сделать рагу на это все дело, навернуть, проиндексировать все этот массив для того, чтобы придумать, как его подкидывать в контекст.  то, что называется Retrieval Augmented Generation.
- [14:03] This chunk occurs during a conversation between Dmitriy Grankin and Windy Hell, where they discuss the challenges and experiences related to collaborating with influencers and bloggers for marketing purposes. Dmitriy reflects on a previous partnership with a Brazilian influencer and seeks to clarify what Windy needs to initiate a new collaboration, while also inquiring about the timeline for attracting new bloggers or users.
  > Dmitriy Grankin:  У меня был договор, у меня была девушка, которая занималась его поиском этого чувака нашла вы на этом и закончилась она такая в облаках летающая бразильянка Она пропала?  Нет, она не пропала, просто она как-то так, что-то у нее.  Мы не смогли продолжить просто.  но она не в состоянии странных стран не вообще португальцы бразильцы они вообще Вообще от жизни по-другому кайф получаю.  Не обязательно работать и зарабатывать.  Вот, так что ты мне сформулируй, что тебе от меня надо, чтобы начать, и давай попробуем, потому что я не знаю, что. Сформулируй, если еще.  какие-то есть условия, которые для тебя важны. И давай пробовать.  А сколько  в ближайшее время ходится?  темблогеров привлечь или пользователей?
- [03:48] This chunk is part of a conversation between Dmitriy Grankin and Windy Hell, where Dmitriy is demonstrating his product that provides real-time transcription of calls, highlighting its ability to identify important keywords and facilitate inquiries about the conversation's content, specifically in the context of influencer marketing.
  > Dmitriy Grankin:  Да, в общем, то, что происходит во время звонка, это real-time transcription.  определяет важные слова которые которые можно кликнуть вот это слово рынок я на слово рынок говорит Ты упоминала рынок в контексте инфлюенс-маркетинга, да?  Вот, можно что угодно спросить, там, что делает Дмитрий, что?  Просто написать, что я делаю, потому что он знает, что я Дмитрий.
- [09:31] In this section of the conversation, Dmitriy Grankin discusses potential collaboration opportunities with Windy Hell, specifically focusing on attracting 30-40 bloggers to promote his product through partnership or free trial arrangements.
  > Dmitriy Grankin:  Какая-то идея, какое-то предложение, что мы можем обсудить, что мы можем сделать.  Могу попробовать блогеров 30-40 на таких же условиях привлечь.  А какие такие же?  Ну, по партнёрке или за бесплатный триал попробовать.  По партнёрке или бесплатный триал.

##  [28] - December 18, 2024 13:55

- [08:49] This chunk is part of a discussion about organizing the "Semana Lixo Zero" event, where Luiza Duarte expresses her concerns about a colleague named Vini's ability to contribute effectively and suggests that Mal would be a better choice for creating a presentation. The conversation revolves around planning and coordinating efforts for upcoming environmental initiatives.
  > Luiza Duarte:  Eu sei que o Vini, ele tá afim de entrar no Semana Lixo Zero e tudo mais, só que eu não confio nele pra ele ser rápido o suficiente da forma como a gente precisa. Então eu acho que para agora, como a gente já quer pegar essas empresas logo agora em janeiro, fevereiro, a melhor pessoa para montar uma A apresentação bem bonita é a Mal, se ela topar.  Ela toparia, ela toparia.  A gente está até conversando com ela para ela fazer a do Rumo ao Lixo Zero.  Daqui a pouco a gente já engata o Rumo ao Lixo Zero, depois a Semana do Lixo Zero.  Que a Maureen, ela não vai precisar, ela não precisa de todo o conceito, tudo aquela coisa desmastigada que o Vini pede, sabe?  Ah, sim, sim.  Pois é.


In [11]:
chat_manager = UnifiedChatManager(
        qdrant_engine=qdrant_engine,
        es_engine=es_engine
    )

In [12]:
chat_manager.messages

[]

In [13]:

@jupyter_stream_output(markdown=True)
async def unified_chat(
    query: str,
    meeting_id: Optional[UUID] = None,
    entities: Optional[List[str]] = None,
    thread_id: Optional[str] = None,
    temperature: float = 0.7
):
    """
    Unified chat function supporting both single-meeting and multi-meeting search
    """
    
    async for result in chat_manager.chat(
        user_id=user_id,
        query=query,
        meeting_id=meeting_id,
        entities=entities,
        thread_id=thread_id,
        temperature=temperature
    ):
        if 'chunk' in result:
            yield result['chunk']
        elif 'linked_output' in result:
            yield result

In [14]:
thread_id=None

In [18]:
r = await unified_chat(
    query="put it all together",
    thread_id=thread_id
)
thread_id = r['thread_id']

Certainly! Here’s a comprehensive overview of VEXA, integrating all the discussed aspects:

---

### VEXA Overview

**VEXA** is an innovative meeting note-taking tool designed to enhance productivity and knowledge management during online meetings. It leverages advanced features to provide users with a seamless experience, making it a valuable asset for individuals and organizations alike.

#### Key Features:

1. **Real-Time Transcription**: VEXA captures conversations in real-time, allowing users to focus on discussions without the distraction of note-taking. This feature ensures that important points are recorded accurately.

2. **Multilingual Capabilities**: The tool can automatically switch between languages during meetings, accommodating diverse teams and international participants. This flexibility enhances communication and inclusivity.

3. **Knowledge Building**: VEXA enables users to build a repository of knowledge from their meetings. Users can access transcripts, derive insights, and follow up on action points, making it easier to retain and utilize information.

4. **User-Friendly Dashboard**: The newly introduced dashboard organizes insights from meetings, allowing users to research specific topics or individuals mentioned in discussions. This feature enhances user engagement and facilitates knowledge sharing.

5. **Integration Potential**: VEXA is exploring integrations with platforms like Google Meet and Microsoft Teams to streamline user experience and expand its reach within organizations.

6. **User Feedback and Iteration**: Continuous improvement is a priority for VEXA. Gathering user feedback helps inform updates and feature enhancements, ensuring the tool meets user needs effectively.

7. **Content Creation and SEO**: VEXA aims to create content that addresses specific user needs, such as acquiring transcripts from YouTube. This strategy not only attracts organic traffic but also positions VEXA as a solution to common problems.

8. **Partnerships and Collaborations**: Exploring partnerships with complementary tools can enhance VEXA's capabilities. Collaborations can create comprehensive solutions that benefit users.

9. **Marketing Strategies**: Engaging micro-influencers and utilizing social media for user testimonials can amplify VEXA's reach. Targeted outreach can help build brand awareness and attract new users.

10. **Analytics and User Insights**: Implementing analytics tools to track user behavior provides insights into feature usage and user engagement, guiding future developments.

11. **Security and Privacy**: Addressing security and privacy concerns is essential, especially for enterprise users. Offering solutions like deploying VEXA on client infrastructure can alleviate data security concerns.

#### Strategic Focus:

- **Dual Market Approach**: VEXA targets both individual users and companies, allowing for a strong user base while addressing the needs of larger organizations.
- **Influencer Marketing**: Leveraging influencers to promote VEXA, particularly in markets like Brazil, can enhance brand visibility and user engagement.
- **Continuous Improvement**: By iterating on user feedback and exploring new features, VEXA can adapt to changing user needs and market demands.

---

In summary, VEXA is positioned as a powerful tool for enhancing meeting productivity and knowledge management. Its unique features, strategic marketing approaches, and focus on user needs can significantly contribute to its growth and user adoption. 

If you have any further questions or specific areas you'd like to explore, feel free to ask!

In [20]:
r['linked_output']

'In the recent meeting between you, Dmitriy Grankin, and Jasper Vanuytrecht, several key topics were discussed regarding your product related to meeting summaries and knowledge management. Here’s a summary of the main points:\n\n1. **Product Features and User Base**: You highlighted the functionality of your product, which includes a dashboard that provides summaries and a chat interface similar to ChatGPT. This allows users to generate insights based on specific meetings or a collection of meetings. You mentioned that the product is currently in a free trial phase, attracting a stable user base with minimal marketing efforts ([1](/meeting/2a40c363-c52b-4f89-8719-e0be87591485)).\n\n2. **Sales and Marketing Strategy**: Jasper emphasized the importance of identifying target customers and crafting effective outreach messages. He suggested that a strategic approach to sales and marketing is crucial for growth, particularly for tools addressing enterprise privacy and knowledge management ch

In [13]:
meeting_ids = ['q','w']

In [14]:
url_dict = {
    idx+1: f"/meeting/{mid}"
    for idx, mid in enumerate(meeting_ids) if mid
} if meeting_ids else {}

In [15]:
url_dict

{1: '/meeting/q', 2: '/meeting/w'}

In [16]:
import re

In [17]:
async def embed_links(text: str, url_dict: dict) -> str:
    # Find all numeric references in the text
    references = re.findall(r'\[(\d+)\]', text)
    
    # Replace each reference with the corresponding URL based on position
    for idx, ref in enumerate(references, 1):
        if idx in url_dict:
            text = text.replace(f'[{ref}]', f'[{ref}]({url_dict[idx]})')
    return text

In [18]:
await embed_links(r['linked_output'],url_dict)

'In the recent meeting between you, Dmitriy Grankin, and Jasper Vanuytrecht, several key topics were discussed regarding your product related to meeting summaries and knowledge management. Here’s a summary of the main points:\n\n1. **Product Features and User Base**: You highlighted the functionality of your product, which includes a dashboard that provides summaries and a chat interface similar to ChatGPT. This allows users to generate insights based on specific meetings or a collection of meetings. You mentioned that the product is currently in a free trial phase, attracting a stable user base with minimal marketing efforts ([1](/meeting/q)(/meeting/2a40c363-c52b-4f89-8719-e0be87591485)).\n\n2. **Sales and Marketing Strategy**: Jasper emphasized the importance of identifying target customers and crafting effective outreach messages. He suggested that a strategic approach to sales and marketing is crucial for growth, particularly for tools addressing enterprise privacy and knowledge m

In [19]:
chat_manager = SearchChatManager()

NameError: name 'SearchChatManager' is not defined

In [65]:
from helpers import jupyter_stream_output
from typing import Optional

In [ ]:
from typing import Optional
from helpers import jupyter_stream_output
from search import SearchAssistant

# Initialize managers
search_assistant = SearchAssistant()
chat_manager = SearchChatManager()

In [ ]:

@jupyter_stream_output(markdown=True)
async def stream_chat(user_id: str, query: str, thread_id: Optional[str] = None):
    async for result in chat_manager.chat(
        user_id=user_id,
        query=query,
        search_assistant=search_assistant,
        thread_id=thread_id
    ):
        if 'chunk' in result:
            yield result['chunk']
        elif 'output' in result:
            yield result['output']

# Usage
await stream_chat(user_id=user_id, query="андрей")

In [ ]:

# Call the decorated function
await stream_chat(user_id=user_id, query="маркетинг")

In [ ]:
await stream_chat(user_id=user_id,query="маркетинг")

In [ ]:
SearchChatManager

In [21]:
r = await ParsedSearchRequest.parse_request("маркетинг")
queries = r.search_queries

In [ ]:
queries

In [40]:
search_results = [
            await search_assistant.search(q.query, user_id=user_id) 
            for q in queries
        ]

In [41]:
import pandas as pd

In [42]:
search_results = pd.concat(search_results)
search_results = search_results.drop(columns=['vector_scores', 'exact_matches'])\
    .drop_duplicates(subset=['topic_name', 'speaker_name', 'summary', 'details', 'meeting_id'])

# Prepare context using search assistant's methods
prepared_df, indexed_meetings = search_assistant.prep_context(search_results)

In [43]:
columns_to_drop = ['timestamp', 'vector_scores', 'exact_matches', 'source', 'score', 'meeting_id']
existing_columns = [col for col in columns_to_drop if col in prepared_df.columns]
context_prepared = prepared_df.drop(columns=existing_columns)

context = context_prepared.to_markdown(index=False) if not prepared_df.empty else "No relevant context found."
        

In [45]:
context = await context_provider._get_raw_context(user_id=user_id,query="маркетинг")

In [ ]:
context

In [ ]:
context_provider.url_dict

In [ ]:
await search_assistant.search(query="vexa",user_id=user_id)

In [ ]:
context = await context_provider.get_context(user_id=user_id,query="vexa")

In [ ]:
context

In [ ]:

meetings = await vexa.get_meetings()

In [6]:
from chat import MeetingContextProvider,MeetingChatManager

In [7]:
from psql_helpers import get_session,get_meeting_by_id

In [8]:
from helpers import jupyter_stream_output

In [21]:
meeting_id = 'dbf8ad27-649c-4a16-b5a1-5ab3db002bd9'

In [ ]:
from uuid import UUID
meeting_id = UUID(meeting_id)

In [ ]:
meeting_id

In [23]:
query = 'summary'

In [24]:
thread_id = None

In [25]:
from prompts import Prompts
prompts = Prompts()
prompt = prompts.single_meeting_2711

In [29]:
context_provider = MeetingContextProvider([meeting_id])

In [ ]:
async with get_session() as session:
    context = await context_provider.get_context(session=session)
context


In [ ]:
@jupyter_stream_output(markdown=True)
async def chat_with_meeting(thread_id, user_id, meeting_id, query):
    async with get_session() as session:
        context_provider = MeetingContextProvider([meeting_id])
        meeting_chat_manager = MeetingChatManager(session, context_provider=context_provider)
        async for response in meeting_chat_manager.chat(
            meeting_ids=[meeting_id],
            user_id=user_id,
            query=query,
            thread_id=thread_id,
            temperature=0.7,
            prompt=prompts.single_meeting_2711
        ):
            yield response
# Usage
response = await chat_with_meeting(
    thread_id=thread_id,
    user_id='ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
    meeting_id=meeting_id,
    query="summare meeting"
)

In [ ]:
meeting_

In [ ]:
await meeting_chat_manager.chat(user_id=user_id,query="What was the main topic of the meeting?",meeting_ids=[meeting_id])